# Data

In [3]:
import numpy as np
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

import sys
import os
root_dir = r'D:\Dataset\Hyperspectral\Blueberry'
tmp_save_dir = r'D:\test'
sys.path.insert(0, root_dir)

import matplotlib.pyplot as plt
from spectral import *
from  pathlib import Path
import cv2 as cv
import cv2
import numpy as np
import pandas as pd
import plotly.express as px
import time 

from modeling_spectral_data import load_hyper_spectral_data, good_dir, bad_dir, good_dir1, bad_dir1
from modeling_spectral_data import save_dir_im_data_good, save_dir_im_data_bad, save_dir_im_data_good1, save_dir_im_data_bad1
from modeling_spectral_data import get_calibrated_image, find_nearest

"""
ref_data
1st day 1800

2nd day 1600
    ref_data[:1600, :, 0]
"""

hyper_spectral_data_dict, data_idx_dict = load_hyper_spectral_data()
ref_data = hyper_spectral_data_dict['ref_data']

def log_xalg(*info, log_path=None, show=True, end=None, flush=False):
    if show:
        if end is not None:
            print(*info, end=end)
        else:
            print(*info)
    if log_path is not None:
        if flush:
            f_log = open(log_path, 'w', encoding="utf-8")
        else:
            f_log = open(log_path, 'a', encoding="utf-8")
            print(*info, file=f_log)
        f_log.close()

def plotly_fig_config(fig, title):
    fig.update_layout({
        'plot_bgcolor': 'rgba(255,255,255,1)',
        'paper_bgcolor': 'rgba(255,255,255,1)'
    })
    fig.update_layout(
        xaxis=dict(
            showline=True,
            showgrid=False,
            showticklabels=True,
            zeroline=False,
            #linecolor='rgb(204, 204, 204)',
            linewidth=2,
            ticks='outside',
            tickfont=dict(
                family='Times New Roman',
                size=20,
                color='rgb(0, 0, 0)',
            ),
        ),
        yaxis=dict(
            showline=True,
            showgrid=False,
            showticklabels=True,
            zeroline=False,
            #linecolor='rgb(204, 204, 204)',
            tickfont=dict(
                family='Times New Roman',
                size=20,
                color='rgb(0, 0, 0)',
            ),
        ),
        autosize=False,
        #automargin=True,
        margin=dict(
            autoexpand=False,
            l=50,
            r=50,
            t=50,
            b=40,
        ),
        showlegend=False,
        #plot_bgcolor='white'
    )
    fig.add_shape(
            # Rectangle with reference to the plot
                type="rect",
                xref="paper",
                yref="paper",
                x0=0,
                y0=0,
                x1=1.0,
                y1=1.0,
                line=dict(
                    color="black",
                    width=1,
                )
            )
    fig.update_layout(title_text='<b>'+title+'</b>', title_x=0.5, title_font_family="Times New Roman", title_font_size=30)
    return fig

def show_statistics(x, reshape=False):
    xxx = np.array(x)
    if reshape:
        xxx = xxx.reshape(-1)
    df_describe = pd.DataFrame(xxx)
    print(df_describe.describe())

	Data Source:   'D:\Dataset\HyperspectralBlueberry\BlueberryScansforDestructiveTesting06132022\GoodBlueberryScans\Good Blueberry 1-42.bil'
	# Rows:           1800
	# Samples:        1600
	# Bands:           462
	Interleave:        BIL
	Quantization:  16 bits
	Data format:    uint16
	Data Source:   'D:\Dataset\HyperspectralBlueberry\BlueberryScansforDestructiveTesting06132022\BadBlueberryScans\Bad Blueberry 1-42.bil'
	# Rows:           1800
	# Samples:        1600
	# Bands:           462
	Interleave:        BIL
	Quantization:  16 bits
	Data format:    uint16


In [4]:
import spectral
spectral.__version__
import cv2
cv2.__version__

'4.8.0'

## assign dataset

In [5]:
"""
not use cropping now
"""
#data_good
#data_bad
data_name = 'data_good'
data = data_idx_dict[data_name]['data']
sample_start_idx = data_idx_dict[data_name]['start_idx']
save_name = data_idx_dict[data_name]['name']

wavelengths = data.metadata['wavelength']
wavelengths = [float(x) for x in wavelengths]
data_num = len(wavelengths)
title_window = 'x'

print(data_name, save_name, data)

data_good good_1_42 	Data Source:   'D:\Dataset\HyperspectralBlueberry\BlueberryScansforDestructiveTesting06132022\GoodBlueberryScans\Good Blueberry 1-42.bil'
	# Rows:           1800
	# Samples:        1600
	# Bands:           462
	Interleave:        BIL
	Quantization:  16 bits
	Data format:    uint16


In [6]:
waitKey_time = 1
save_showed_im = 0
global_im_idx = 0

In [7]:
"""
imshow(winname, mat) -> None
. The function may scale the image, depending on its depth:
. - If the image is 8-bit unsigned, it is displayed as is.
. - If the image is 16-bit unsigned or 32-bit integer, the pixels are divided by 256. 
    That is, the value range [0,255\*256] is mapped to [0,255].
. - If the image is 32-bit or 64-bit floating-point, the pixel values are multiplied by 255. That is, the
.   value range [0,1] is mapped to [0,255].
"""
def im_resize_show(image, width = 600, height = None, inter = cv2.INTER_AREA, waitKey_time=waitKey_time, name='x', save_showed_im=save_showed_im):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv.resize(image, dim, interpolation = inter)
    cv.namedWindow(name, 1)
    cv.imshow(name, resized)
    save_dir = os.path.join(root_dir, 'results')
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    global global_im_idx
    print('global_im_idx:', global_im_idx, 'image.dtype:', image.dtype, 'image min:', image.min(), 'image max:', image.max())
    if image.dtype != np.uint8:
        print('clamp image range')
        image_norm = image*255
        print('global_im_idx:', global_im_idx, 'image.dtype:', image_norm.dtype, 'image min:', image_norm.min(), 'image max:', image_norm.max())
        idxes = np.where(image_norm > 255)
        image_norm[idxes[0], idxes[1]] = 255
        print('global_im_idx:', global_im_idx, 'image.dtype:', image_norm.dtype, 'image min:', image_norm.min(), 'image max:', image_norm.max())

        idxes = np.where(image_norm < 0)
        image_norm[idxes[0], idxes[1]] = 0
        image_norm = np.round(image_norm).astype(np.uint8)
        print('global_im_idx:', global_im_idx, 'image.dtype:', image_norm.dtype, 'image min:', image_norm.min(), 'image max:', image_norm.max())
        image_norm=cv.normalize(image_norm,  None, 0, 255, cv.NORM_MINMAX)
    else:
        image_norm=cv.normalize(image,  None, 0, 255, cv.NORM_MINMAX)
    if save_showed_im:
        save_path = os.path.join(save_dir, name+'_'+str(global_im_idx)+'.jpg')
        print('save to', save_path)
        cv.imwrite(save_path, image_norm)
    global_im_idx += 1
    print('waitKey_time', waitKey_time)
    res = cv.waitKey(waitKey_time)

In [6]:
"""
460/550/640
435.8/546.1/700

show keep aspect ratio
700 nm (red), 546.1 nm (green) and 435.8 nm (blue).
"""
# 700 nm (red), 546.1 nm (green) and 435.8 nm (blue)
b = get_calibrated_image(435.8, wavelengths, data, ref_data)[:, :, None]
g = get_calibrated_image(546.1, wavelengths, data, ref_data)[:, :, None]
r = get_calibrated_image(700, wavelengths, data, ref_data)[:, :, None]

rgb = np.concatenate([b,g,r], 2).squeeze()

#rgb1 = rgb/rgb.max()

#rgb = (rgb - rgb.min()) / (rgb.max() - rgb.min())
#rgb = np.round(rgb*255).astype(np.uint8)
#im_resize_show(rgb)

#import plotly.express as px
#fig = px.imshow(rgb)
#fig.show()

rgb_quantization = rgb*255
idxes = np.where(rgb_quantization > 255)
rgb_quantization[idxes[0], idxes[1]] = 255
rgb_quantization = np.round(rgb_quantization).astype(np.uint8)
#diff_im_optimize = cv.cvtColor(rgb_quantization, cv.COLOR_BGR2GRAY)

im_resize_show(rgb_quantization)

435.88 32
545.54 115
700.03 231
global_im_idx: 0 image.dtype: uint8 image min: 0 image max: 255
waitKey_time 1


In [7]:
#"""
#show datacube
#"""
#from spectral import open_image
##view = imshow(b[:,:,0,0])
##from spectral.graphics import view_cube
#b = get_calibrated_image(435.8, wavelengths, data, ref_data)[:, :, None]
#g = get_calibrated_image(546.1, wavelengths, data, ref_data)[:, :, None]
#r = get_calibrated_image(700, wavelengths, data, ref_data)[:, :, None]
#im1 = get_calibrated_image(400, wavelengths, data, ref_data)[:, :, None]
#im2 = get_calibrated_image(500, wavelengths, data, ref_data)[:, :, None]
#im3 = get_calibrated_image(600, wavelengths, data, ref_data)[:, :, None]
#im4 = get_calibrated_image(750, wavelengths, data, ref_data)[:, :, None]
#im5 = get_calibrated_image(800, wavelengths, data, ref_data)[:, :, None]
#im6 = get_calibrated_image(900, wavelengths, data, ref_data)[:, :, None]
#data_cube = np.concatenate([r,g,b, im1, im2, im3, im4, im5, im6], 2).squeeze()
##data_cube = data_cube.transpose(2,0,1)
#print(data_cube.shape)
##data=rgb_quantization
##import spectral
##import wx
##app=wx.App()
##spectral.settings.WX_GL_DEPTH_SIZE = 16
##spectral.view_cube(data_cube)
##app.MainLoop()
##data.shape
##open_image(r'D:\BoyangDeng\BlueberryClassification\datasets\BlueberryScansforDestructiveTesting06132022\GoodBlueberryScans\Good Blueberry 1-42.bil')

In [8]:
"""
data cube
"""
#data.shape
#right = data[:, -1, :].squeeze()
#top = data[0, :, :].squeeze()
#front = data[:, :, 5].squeeze()
#print(right.shape, top.shape, front.shape)

#pad=np.zeros((200,462))
#top=np.concatenate([pad, top], 0)

##im = right
#front = front
#front=cv.normalize(front,  None, 0, 255, cv.NORM_MINMAX)
#front = np.round(front).astype(np.uint8)

#top = top
#top=cv.normalize(top,  None, 0, 255, cv.NORM_MINMAX)
#top = np.round(top).astype(np.uint8)

#right = right
#right=cv.normalize(right,  None, 0, 255, cv.NORM_MINMAX)
#right = np.round(right).astype(np.uint8)

#tot=np.concatenate([front, top, right], 1)

#im_resize_show(tot, save_showed_im=1)

'\ndata cube\n'

In [9]:
#im0 = get_calibrated_image(428, wavelengths, data, ref_data)
im1 = get_calibrated_image(680, wavelengths, data, ref_data)
im2 = get_calibrated_image(900, wavelengths, data, ref_data)

#show_im(im2-im1)
diff_im = im2-im1
diff_im = diff_im[:, :, 0]
print(diff_im.min(), diff_im.max())
print('diff_im:', diff_im.dtype)

im_resize_show(diff_im)

#import plotly.express as px
#fig = px.imshow(diff_im)
#fig.show()

679.96 216


900.58 380
-0.5947371006753429 1.1656922697337206
diff_im: float64
global_im_idx: 1 image.dtype: float64 image min: -0.5947371006753429 image max: 1.1656922697337206
clamp image range
global_im_idx: 1 image.dtype: float64 image min: -151.65796067221245 image max: 297.2515287820988
global_im_idx: 1 image.dtype: float64 image min: -151.65796067221245 image max: 255.0
global_im_idx: 1 image.dtype: uint8 image min: 0 image max: 255
waitKey_time 1


In [10]:
"""
not use
background part pixels is randomly, so subtraction may be not near to zero and have some negative value to -0.5
foreground pixels subtraction max pixels is around 1.1 
so this fomula will decrease the contrast
"""
#diff_im_norm = cv.normalize(diff_im, diff_im, 0,255, cv.NORM_MINMAX)

#diff_im_norm = (diff_im - diff_im.min()) / (diff_im.max() - diff_im.min())
#diff_im_norm = np.round(diff_im_norm*255).astype(np.uint8)

#import plotly.express as px
#fig = px.imshow(diff_im_norm)
#fig.show()

'\nnot use\nbackground part pixels is randomly, so subtraction may be not near to zero and have some negative value to -0.5\nforeground pixels subtraction max pixels is around 1.1 \nso this fomula will decrease the contrast\n'

In [11]:
idxes = np.where(diff_im < 0.0)
diff_im_optimize = diff_im.copy()
diff_im_optimize[idxes[0], idxes[1]] = 0

#idxes = np.where(diff_im_optimize > 1.0)
#diff_im_optimize[idxes[0], idxes[1]] = 1.0

diff_im_optimize = diff_im_optimize / diff_im_optimize.max()
print(diff_im_optimize.min(), diff_im_optimize.max())
diff_im_optimize = np.round(diff_im_optimize*255).astype(np.uint8)

im_resize_show(diff_im_optimize)

#import plotly.express as px
#fig = px.imshow(diff_im_optimize)
#fig.show()

0.0 1.0
global_im_idx: 2 image.dtype: uint8 image min: 0 image max: 255
waitKey_time 1


In [12]:
"""
maybe use in special cases to connect boundary of defect samples
equalizeHist cause high frequency noise in background
"""
diff_im_optimize_eh = cv.equalizeHist(diff_im_optimize)
im_resize_show(diff_im_optimize_eh)

#import plotly.express as px
#fig = px.imshow(diff_im_optimize)
#fig.write_image(r"F:\test\diff_im_optimize.png", width=1400, height=1000)

##pip install -U kaleido
#fig = px.imshow(diff_im_optimize_eh)
#fig.write_image(r"F:\test\diff_im_optimize_eh.png", width=1400, height=1000)

#fig.write_html(r"F:\test\xxx.html")

global_im_idx: 3 image.dtype: uint8 image min: 0 image max: 255
waitKey_time 1


In [13]:
from skimage.filters import threshold_yen, threshold_otsu, try_all_threshold
#thresh = threshold_yen(diff_im_optimize)
#ret, diff_im_optimize_t = cv.threshold(diff_im_optimize, thresh, 255, cv.THRESH_BINARY)
"""
maybe use in special case to connect boundary of defect samples
"""

thresh = threshold_yen(diff_im_optimize_eh)
ret, diff_im_optimize_t = cv.threshold(diff_im_optimize_eh, thresh, 255, cv.THRESH_BINARY)
ret, diff_im_optimize_t = cv.threshold(diff_im_optimize_eh, thresh-50, 255, cv.THRESH_BINARY)

print('thresh:', thresh)
#diff_im_optimize_t = diff_im_optimize > thresh
#diff_im_optimize_t = diff_im_optimize_t.astype(np.uint8)*255

im_resize_show(diff_im_optimize_t)

#import plotly.express as px
#fig = px.imshow(diff_im_optimize_t)
#fig.write_image(r"F:\test\diff_im_optimize_t.png", width=1400, height=1000)

thresh: 145
global_im_idx: 4 image.dtype: uint8 image min: 0 image max: 255
waitKey_time 1


In [14]:
# for convinenent use larger filter
#kernel=cv.getStructuringElement(cv.MORPH_RECT,(20,20))
kernel=cv.getStructuringElement(cv.MORPH_RECT,(5,5))
#kernel=cv.getStructuringElement(cv.MORPH_RECT,(3,3))
iterations = 1
eroded=cv.erode(diff_im_optimize_t,kernel,iterations=iterations)
kernel=cv.getStructuringElement(cv.MORPH_RECT,(3,3))

# 4/2
iterations = 2
diff_im_optimize_open=cv.dilate(eroded,kernel, iterations=iterations)

"""
maybe use in special case to connect boundary of defect samples
"""
iterations = 8
diff_im_optimize_open=cv.dilate(diff_im_optimize_open,kernel, iterations=iterations)
diff_im_optimize_open=cv.erode(diff_im_optimize_open,kernel, iterations=iterations)

#iterations = 4
#diff_im_optimize_open = cv.erode(diff_im_optimize_open, kernel, iterations=iterations)

im_resize_show(diff_im_optimize_open)

#import plotly.express as px
#fig = px.imshow(diff_im_optimize_open)
#fig.write_image(r"F:\test\diff_im_optimize_open.png", width=1400, height=1000)

global_im_idx: 5 image.dtype: uint8 image min: 0 image max: 255
waitKey_time 1


## filter the metal plate

### crop

In [15]:
#crop_y_start = data_idx_dict[data_name]['crop_y_start']
#crop_x_start = data_idx_dict[data_name]['crop_x_start']

#diff_im_optimize_open_crop = diff_im_optimize_open[crop_y_start:, crop_x_start:]
#cv.namedWindow(title_window, 0)
#cv.imshow(title_window, diff_im_optimize_open_crop)
#cv.waitKey(0)

In [16]:
#rgb_crop = rgb[crop_y_start:, crop_x_start:]

In [17]:
#diff_crop = diff_im_optimize[crop_y_start:, crop_x_start:]
#diff_crop_3_channels = diff_crop[:,:,None]
#diff_crop_3_channels = np.concatenate([diff_crop_3_channels,diff_crop_3_channels,diff_crop_3_channels], 2).squeeze()
#diff_crop_3_channels.shape

### circle filter

In [18]:
diff_3_channels = diff_im_optimize[:,:,None]
diff_3_channels = np.concatenate([diff_3_channels,diff_3_channels,diff_3_channels], 2).squeeze()
im_resize_show(diff_3_channels)

#import plotly.express as px
#fig = px.imshow(diff_im_optimize)
#fig.show()

global_im_idx: 6 image.dtype: uint8 image min: 0 image max: 255
waitKey_time 1


In [19]:
from modeling_filter_metal_plate import detect_circle
find_circle_list, img_circle_bgr = detect_circle(diff_im_optimize_open)
im_resize_show(img_circle_bgr)

(1, 137, 3)
global_im_idx: 7 image.dtype: uint8 image min: 0 image max: 255
waitKey_time 1


## find contour

In [20]:
im_copy = np.zeros_like(diff_im_optimize_open)
res = cv.findContours(image=diff_im_optimize_open, mode=cv.RETR_EXTERNAL, method=cv.CHAIN_APPROX_SIMPLE)
_, contours, hierarchy = res
print(len(contours))

47


In [21]:
valid_contours = []
im_circles = np.zeros_like(diff_im_optimize_open)

for circle in find_circle_list:
    x,y,r = circle
    cv.circle(im_circles, (x,y), r, 255, -1)

im_resize_show(im_circles)

for contour in contours:
    im_one_conotur = np.zeros_like(diff_im_optimize_open)
    cv.drawContours(image=im_one_conotur, contours=[contour], contourIdx=-1,
                    color=255, thickness=-1, lineType=cv.LINE_AA)
    res_im = im_circles * im_one_conotur
    idxes = np.where(res_im > 0)
    if len(idxes[0]) > 0:
        valid_contours.append(contour)
print(len(valid_contours))
contours = valid_contours

global_im_idx: 8 image.dtype: uint8 image min: 0 image max: 255
waitKey_time 1


42


In [ ]:
contours[0]

In [ ]:
rgb_with_contour = rgb.copy()
cv.drawContours(image=rgb_with_contour, contours=contours, contourIdx=-1,
                color=[0.1, 0.5, 0.2], thickness=3, lineType=cv.LINE_AA)
# see the results
im_resize_show(rgb_with_contour)
#import plotly.express as px
#fig = px.imshow(rgb_with_contour*255)
#fig.show()

In [264]:
diff_3_channels_contour = diff_3_channels.copy()
cv.drawContours(image=diff_3_channels_contour, contours=contours, contourIdx=-1,
                color=[10, 200, 50], thickness=2, lineType=cv.LINE_AA)
# see the results
im_resize_show(diff_3_channels_contour)

#import plotly.express as px
#fig = px.imshow(diff_3_channels_contour)
#fig.show()

global_im_idx: 10 image.dtype: uint8 image min: 0 image max: 255
waitKey_time 1


In [265]:
"""
note order should be correct

rgb_with_contour/diff_3_channels_contour
"""
im_copy = rgb_with_contour.copy()
font=cv.FONT_HERSHEY_DUPLEX
center_pts = []
center_refs = []
for i_contour, contour in enumerate(contours):
    x = int(contour[:, :, 0].mean())
    y = int(contour[:, :, 1].mean())
    # base
    #center_refs.append(x+10*y)
    # bad 127-168 / bad 1-42 / others seem all OK
    center_refs.append(x+7*y)
    center_pts.append((x, y))
idx_sorted = np.argsort(center_refs)
# [10, 50, 200] [0.1, 0.3, 0.8]
for i_idx_sorted, idx in enumerate(idx_sorted):
    x, y = center_pts[idx]
    cv.putText(im_copy, str(i_idx_sorted+ 1 + sample_start_idx) , (x-5,y+5), font, 1, [10, 50, 200], 2)
print(len(contours))
im_resize_show(im_copy)

42
global_im_idx: 11 image.dtype: float64 image min: 0.0 image max: 200.0
clamp image range
global_im_idx: 11 image.dtype: float64 image min: 0.0 image max: 51000.0


global_im_idx: 11 image.dtype: float64 image min: 0.0 image max: 255.0
global_im_idx: 11 image.dtype: uint8 image min: 0 image max: 255
waitKey_time 1


## mean spectral value of each band of each sample

In [221]:
import cv2 as cv
from tqdm import tqdm

wavelengths = data.metadata['wavelength']
wavelengths = [float(x) for x in wavelengths]
data_num = len(wavelengths)
hyper_dict = {}
for i_band in tqdm(range(data_num), ncols=80):
    im = data[: , :, i_band]
    ref = ref_data[:im.shape[0] , :, i_band]
    im_corrected = im/ref
    #im_corrected = im_corrected[crop_y_start:, crop_x_start:]
    
    for i_idx_sorted, idx in enumerate(idx_sorted):
        contour = contours[idx]
        mask = np.zeros_like(im_corrected)
        cv.drawContours(mask, [contour], -1, 255,-1)
        pts  = np.where(mask == 255)
        roi = im_corrected[pts[0], pts[1]]
        mean_spectral = round(np.mean(roi), 4)
        if i_idx_sorted not in hyper_dict:
            hyper_dict[i_idx_sorted] = []
        hyper_dict[i_idx_sorted].append(mean_spectral)
hyper_dict.keys()

100%|█████████████████████████████████████████| 462/462 [25:27<00:00,  3.31s/it]


dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41])

In [266]:
save_dir = os.path.join(root_dir, 'mean_data_python')
save_dir = Path(save_dir)
save_dir.mkdir(parents=True, exist_ok=True)
save_path = save_dir / (save_name+'.npy').__str__()
save_path

WindowsPath('D:/BoyangDeng/BlueberryClassification/datasets/mean_data_python/good_1_42.npy')

In [223]:
np.save(save_path, hyper_dict) 
print(save_path)

D:\BoyangDeng\BlueberryClassification\datasets\mean_data_python\bad_1_42.npy


In [267]:
hyper_dict = np.load(save_path,allow_pickle='TRUE').item()
print(save_path)
len(hyper_dict[0])

D:\BoyangDeng\BlueberryClassification\datasets\mean_data_python\good_1_42.npy


462

In [ ]:
from modeling_spectral_data import show_mean_spectral_data
show_mean_spectral_data(hyper_dict, 10, wavelengths)

In [ ]:
from modeling_spectral_data import show_hyper_dict
show_hyper_dict(hyper_dict, wavelengths)

## save each sample each band data

In [268]:
save_dir_origin_data_good = good_dir / 'origin_data'
save_dir_im_data_good = good_dir / 'im_data_local'

save_dir_origin_data_bad = bad_dir / 'origin_data'
save_dir_im_data_bad = bad_dir / 'im_data_local'

In [269]:
root_dir = data_idx_dict[data_name]['data_dir']
save_dir_origin_data = root_dir / 'origin_data'
save_dir_origin_data.mkdir(parents=True, exist_ok=True)

#save_dir_im_data_global = root_dir / 'im_data_global'
#save_dir_im_data_global.mkdir(parents=True, exist_ok=True)

save_dir_im_data_local = root_dir / 'im_data_local'
save_dir_im_data_local.mkdir(parents=True, exist_ok=True)

In [270]:
import cv2 as cv
from tqdm import tqdm

wavelengths = data.metadata['wavelength']
wavelengths = [float(x) for x in wavelengths]
data_num = len(wavelengths)
hs=[]
ws=[]
for i_band in tqdm(range(data_num), ncols=80):
#for i_band in tqdm(range(0,1), ncols=80):
    im = data[: , :, i_band]
    ref = ref_data[:im.shape[0] , :, i_band]
    im_corrected = im/ref
    #im_corrected = im_corrected[crop_y_start:, crop_x_start:]
    
    for i_idx_sorted, idx in enumerate(idx_sorted):
        contour = contours[idx]
        mask = np.zeros_like(im_corrected)
        cv.drawContours(mask, [contour], -1, 255,-1)
        pts  = np.where(mask == 255)

        y0 = pts[0].min()
        y1 = pts[0].max()
        x0 = pts[1].min()
        x1 = pts[1].max()
        hs.append(y1-y0)
        ws.append(x1-x0)
        #continue

        one_sample = im_corrected[y0:y1+1, x0:x1+1]

        save_dir_origin_data_one_sample = save_dir_origin_data / str(sample_start_idx + i_idx_sorted+1)
        save_dir_origin_data_one_sample.mkdir(parents=True, exist_ok=True)
        save_path = save_dir_origin_data_one_sample / f'{i_band}_{wavelengths[i_band]}.npy'
        np.save(save_path, one_sample)

        #save_dir_im_data_one_sample_global = save_dir_im_data_global / str(sample_start_idx + i_idx_sorted+1)
        #save_dir_im_data_one_sample_global.mkdir(parents=True, exist_ok=True)
        #save_path = save_dir_im_data_one_sample_global / f'{i_band}_{wavelengths[i_band]}.png'
        #highest_t = 3.0
        #one_sample = np.array(one_sample/highest_t*255, dtype=np.uint8)
        #cv.imwrite(save_path.__str__(), one_sample)

        save_dir_im_data_one_sample_local = save_dir_im_data_local / str(sample_start_idx + i_idx_sorted+1)
        save_dir_im_data_one_sample_local.mkdir(parents=True, exist_ok=True)
        save_path = save_dir_im_data_one_sample_local / f'{i_band}_{wavelengths[i_band]}.png'
        one_sample = (one_sample - one_sample.min()) / (one_sample.max() - one_sample.min())
        one_sample = np.array(one_sample*255, dtype=np.uint8)
        cv.imwrite(save_path.__str__(), one_sample)

  0%|                                                   | 0/462 [00:00<?, ?it/s]

100%|█████████████████████████████████████████| 462/462 [17:44<00:00,  2.30s/it]


In [241]:
np.average(hs), np.average(ws)

(130.3095238095238, 150.64285714285714)

# PCA Vision

## show dimension reduction data

In [92]:
import pandas as pd
from modeling_spectral_data import load_spectral_mean_data_xy
x, y = load_spectral_mean_data_xy()
x.shape

(420, 462)

In [93]:
#from modeling_spectral_data import load_spectral_mean_data
#hyper_dict_good_tot, hyper_dict_bad_tot = load_spectral_mean_data()
#from modeling_spectral_data import show_hyper_dict
#show_hyper_dict(hyper_dict_bad_tot, wavelengths)

In [94]:
from sklearn.preprocessing import StandardScaler

x_norm = StandardScaler().fit_transform(x)

In [103]:
x.shape, spatial_features_norm.shape
x = spatial_features_norm

In [104]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['PC1', 'PC2'])
finalDf = pd.concat([principalDf, pd.Series(y, name='class')], axis = 1)

In [105]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')

#font = {'family' : 'normal',
#        'weight' : 'normal',
#        'size'   : 12}

plt.rcParams["font.family"] = "Times New Roman"
#fig = plt.figure(figsize = (8,8))
fig = plt.figure()
ax = fig.add_subplot(1,1,1) 
# ax.set_xlabel('PC1 (85.9%)', fontsize = 16)
# ax.set_ylabel('PC2 (7.0%)', fontsize = 16)
ax.set_xlabel('PC1 (30.2%)', fontsize = 16)
ax.set_ylabel('PC2 (16.0%)', fontsize = 16)
#ax.set_title('PCA with two components', fontsize = 16)
ax.xaxis.labelpad = 5
ax.yaxis.labelpad = 0
#pl.xlabel("...", labelpad=20)
target_labels = ['Sound Blueberry', 'Defective Blueberry']
targets = [0, 1]
colors = ['#FFD300', '#8CA77B']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['class'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'PC1']
               , finalDf.loc[indicesToKeep, 'PC2']
               , c = color
               , s = 50)
ax.legend(target_labels, fontsize = 16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
ax.tick_params(axis="y",direction="in")
#ax.grid()
fig.tight_layout()
plt.rcParams['figure.dpi'] = 800
plt.rcParams['savefig.dpi'] = 800
plt.subplots_adjust(left=0.11, right=0.95, top=0.94, bottom=0.12)
#plt.show(block=True)
plt.savefig(r'D:\test\test.png', bbox_inches='tight')

In [97]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['PC1', 'PC2', 'PC3'])
finalDf = pd.concat([principalDf, pd.Series(y, name='class')], axis = 1)

In [106]:
pca.explained_variance_ratio_

array([0.3022, 0.16  ])

In [11]:
import matplotlib
matplotlib.use('TkAgg')

font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 12}

matplotlib.rc('font', **font)
#%matplotlib inline
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax= Axes3D(fig)
ax.set_xlabel('Principal Component 1', fontsize = 12)
ax.set_ylabel('Principal Component 2', fontsize = 12)
ax.set_zlabel('Principal Component 3', fontsize = 12)
ax.set_title('PCA with 3 components')
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 10
target_labels = ['good', 'bad']
targets = [0, 1]
colors = ['g', 'r']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['class'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'PC1']
               , finalDf.loc[indicesToKeep, 'PC2']
               , finalDf.loc[indicesToKeep, 'PC3']
               , c = color
               , s = 50)
ax.legend(target_labels)
ax.grid()
plt.show(block=True)

## detect bands by loadings

In [12]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
X_pca = pca.fit_transform(x)
X_pca.shape

(420, 3)

In [13]:
X_pca_norm = (X_pca - X_pca.mean(0)) / X_pca.std(0)
X_pca_norm.mean(0), X_pca_norm.std(0)

(array([-0.,  0.,  0.]), array([1., 1., 1.]))

In [14]:
for i_show in range(X_pca.shape[1]):
    #if i_show>=2:
    #    break
    plt.plot(X_pca[:, i_show])
plt.show(block=True)

findfont: Font family 'normal' not found.
findfont: Font family 'normal' not found.
findfont: Font family 'normal' not found.
findfont: Font family 'normal' not found.
findfont: Font family 'normal' not found.
findfont: Font family 'normal' not found.
findfont: Font family 'normal' not found.
findfont: Font family 'normal' not found.
findfont: Font family 'normal' not found.
findfont: Font family 'normal' not found.


In [ ]:
#from sklearn.decomposition import IncrementalPCA
#ipca = IncrementalPCA(n_components=20, batch_size=10)
#X_ipca = ipca.fit_transform(x)
#X_ipca.explained_variance_

In [16]:
pca.explained_variance_, pca.explained_variance_ratio_

(array([2.4486, 0.1996, 0.1795]), array([0.8589, 0.07  , 0.063 ]))

In [17]:
loadings_v0 = pca.components_.T
loadings_v0.shape

(462, 3)

In [36]:
pca.components_.shape

(3, 462)

In [18]:
loadings_v1 = pca.components_.T * np.sqrt(pca.explained_variance_)
loadings_v1.shape

(462, 3)

In [19]:
loadings_idxes = np.zeros_like(loadings_v1)
show_num = 3
feature_idx_list = []
for i_feat in range(loadings_v1.shape[1]):
    loadings_idxes[:, i_feat] = np.argsort(loadings_v1[:, i_feat])[::-1]
    for i_show in range(show_num):
        idx = loadings_idxes[i_show, i_feat]
        idx = int(idx)
        ev = pca.explained_variance_[i_feat]
        print(ev, i_feat , idx, loadings_v1[idx, i_feat])
        #if ev > 1e-3 and loadings_v1[idx, i_feat] > 1e-2:
        if ev > 5e-4 and loadings_v1[idx, i_feat] > 1e-3:
            if idx not in feature_idx_list:
                feature_idx_list.append(idx)
print(len(feature_idx_list))

2.448596740037898 0 315 0.11973731156406772
2.448596740037898 0 316 0.11973536372486668
2.448596740037898 0 314 0.11972967619189143
0.19959045101872908 1 254 0.036906699942220206
0.19959045101872908 1 255 0.03689026337304528
0.19959045101872908 1 256 0.036855523336462206
0.1794962545654013 2 237 0.0362675675912167
0.1794962545654013 2 236 0.036246718963353
0.1794962545654013 2 238 0.036210753054797595
9


In [24]:
"""
plot
"""
show_num = 3

font = {'family' : 'Times New Roman',
        'weight' : 'normal',
        'size'   : 16}

matplotlib.rc('font', **font)
#colors = ['#FFD300', '#8CA77B', '#FFC183']
colors = ['#FFD300', '#8CA77B', '#e6194B']
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)
cmap = get_cmap(3*show_num)

loadings_idxes = np.zeros_like(loadings_v1)
feature_idx_list = []
fig=plt.figure()
for i_feat in range(loadings_v1.shape[1]):
    if i_feat >= show_num:
        break
    loadings_per_component = loadings_v1[:, i_feat]
    feature_idx_list.append(loadings_per_component.argmax())
    feature_idx_list.append(loadings_per_component.argmin())

    max_idx = loadings_per_component.argmax()
    max_value = loadings_per_component.max()

    min_idx = loadings_per_component.argmin()
    min_value = loadings_per_component.min()
    print('i_feat:', i_feat, max_idx, max_value, min_idx, min_value)
    plt.plot(wavelengths, loadings_per_component, label=f'PC {i_feat+1}', color=colors[i_feat])

    # color=cmap(3*i_feat)
    plt.scatter(wavelengths[max_idx], max_value, color=colors[i_feat])
    plt.scatter(wavelengths[min_idx], min_value, color=colors[i_feat])
#plt.title('PCA loadings of the first three primary components')
plt.xlabel('Wavelength (nm)', labelpad=5)
plt.ylabel('Loadings of principal components', labelpad=0)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
ax.tick_params(axis="y",direction="in")

#plt.margins(0.2)
plt.legend()
plt.xlim([wavelengths[0], wavelengths[-1]])
fig.tight_layout()
plt.rcParams['figure.dpi'] = 800
plt.rcParams['savefig.dpi'] = 800
plt.subplots_adjust(left=0.11, right=0.95, top=0.94, bottom=0.11)
#plt.show(block=True)
plt.savefig(r'D:\test\PCA_loadings.png', bbox_inches='tight')
#plt.show(block=True)

C:\Users\AFSALab\AppData\Local\Temp\ipykernel_22032\4284100505.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  return plt.cm.get_cmap(name, n)


i_feat: 0 315 0.11973731156406772 156 0.011480691926757816
i_feat: 1 254 0.036906699942220206 15 -0.030539535294593488
i_feat: 2 237 0.0362675675912167 386 -0.02244744554321167


In [20]:
len(feature_idx_list), feature_idx_list

(6, [315, 156, 15, 258, 240, 388])

In [21]:
band_list = []
for i in feature_idx_list:
    band_list.append(wavelengths[i])
band_list1=sorted(band_list)
band_list, band_list1

([812.8, 599.97, 413.52, 736.18, 712.06, 911.41],
 [413.52, 599.97, 712.06, 736.18, 812.8, 911.41])

In [ ]:
!pip install pca

In [19]:
from pca import pca

# Initialize to reduce the data up to the number of componentes that explains 95% of the variance.
model = pca(n_components=0.99)
# Or reduce the data towards 2 PCs
#model = pca(n_components=2)
results = model.fit_transform(x)

[pca] >Column labels are auto-completed.
[pca] >Row labels are auto-completed.
[pca] >The PCA reduction is performed to capture [99.0%] explained variance using the [462] columns of the input data.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.
[pca] >Compute explained variance.
[pca] >Number of components is [3] that covers the [99.00%] explained variance.
[pca] >The PCA reduction is performed on the [462] columns of the input dataframe.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.
[pca] >Outlier detection using Hotelling T2 test with alpha=[0.05] and n_components=[3]
[pca] >Outlier detection using SPE/DmodX with n_std=[2]


In [69]:
results['PC'].shape

(420, 3)

In [ ]:
# Plot explained variance
fig, ax = model.plot()

In [ ]:
# Scatter first 2 PCs
fig, ax = model.scatter()

In [46]:
import matplotlib
matplotlib.use('TkAgg')
#%matplotlib inline

# Make biplot with the number of features
fig, ax = model.biplot(y=y, n_feat=4, PC=[0,1,2], d3=True)
plt.show(block=True)

[pca] >Plot PC1 vs PC2 vs PC3 with loadings.


In [36]:
#find_nearest(wavelengths)
importances = [16, 241, 316]
for i in importances:
    print(wavelengths[i])

414.83
713.4
814.14


# LDA/QDA -> rLDA (and other models)

In [6]:
from sklearn.model_selection import train_test_split
import time
from sklearn.model_selection import train_test_split,cross_val_score,RepeatedStratifiedKFold,GridSearchCV, RandomizedSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import ConfusionMatrixDisplay,precision_score,recall_score,confusion_matrix
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import time

## load data

In [8]:
from modeling_spectral_data import load_spectral_mean_data_xy
x, y, hyper_dict_tot = load_spectral_mean_data_xy(use_part_1=1, use_part_2=1, return_dict=True)
x_norm = (x-x.mean(0))/x.std(0)
X =  pd.DataFrame(x)
X_norm =  pd.DataFrame(x_norm)
label = pd.DataFrame(y, columns=['label'])
X.shape, y.shape

((420, 462), (420,))

In [ ]:
from modeling_spectral_data import show_hyper_dict
show_hyper_dict(hyper_dict_tot, wavelengths, title='Blueberry Spectra')

In [ ]:
#from modeling_spectral_data import load_spectral_mean_data_matlab
#x, y = load_spectral_mean_data_matlab()

from modeling_spectral_data import load_spectral_mean_data
hyper_dict_good_tot, hyper_dict_bad_tot = load_spectral_mean_data(use_part_1=1, use_part_2=1)

from modeling_spectral_data import show_hyper_dict
show_hyper_dict(hyper_dict_bad_tot, wavelengths, title='Blueberry Spectra')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df = pd.concat([X, label], axis= 1)

label = le.fit_transform(df['label'])
label

In [27]:
np.random.seed(int(time.time()))
#random_state = np.random.randint(0, 1e4)
random_state=42
X_train, X_test, y_train, y_test = train_test_split(
    X, label, test_size=0.30, random_state=random_state, shuffle=True, stratify=label)

In [ ]:
import seaborn as sns
sns.countplot(data=df, x='label')
plt.show()

## LDA from lib

In [35]:
"""
can not be used
"""
from sklearn.preprocessing import StandardScaler
x_norm = StandardScaler().fit_transform(X_train)
x_norm.mean(axis=0)[:5], x_norm.std(axis=0)[:5]

(array([ 0., -0., -0., -0.,  0.]), array([1., 1., 1., 1., 1.]))

In [36]:
from sklearn.preprocessing import scale
x_norm1 = scale(X_train)
x_norm1.mean(axis=0)[:5], x_norm1.std(axis=0)[:5]

(array([ 0., -0., -0., -0.,  0.]), array([1., 1., 1., 1., 1.]))

In [188]:
#X_train = x_norm1

In [37]:
X_test = scale(X_test)
X_test.mean(axis=0)[:5], X_test.std(axis=0)[:5]

(array([-0., -0.,  0., -0.,  0.]), array([1., 1., 1., 1., 1.]))

In [ ]:
#LDA.fit_transform(X_train,y_train)
LDA = LinearDiscriminantAnalysis()
LDA.fit(X_train,y_train)
predictions = LDA.predict(X_test)
ConfusionMatrixDisplay.from_predictions(y_test, predictions)
tn, fp, fn, tp = confusion_matrix(list(y_test.values.squeeze()), list(predictions), labels=[0, 1]).ravel()
#tn, fp, fn, tp = confusion_matrix(list(y_test), list(predictions), labels=[0, 1]).ravel()

print('True Positive :', tp)
print('True Negative :', tn)
print('False Positive :', fp)
print('False Negative :', fn)
print("Precision score", precision_score(y_test, predictions))
print("Recall score", recall_score(y_test, predictions))
plt.show(block=True)


In [294]:
1-11/126

0.9126984126984127

In [131]:
train_predictions = LDA.predict(X_train)
ConfusionMatrixDisplay.from_predictions(y_train, train_predictions)
tn, fp, fn, tp = confusion_matrix(list(y_train.values.squeeze()), list(train_predictions), labels=[0, 1]).ravel()
plt.show(block=True)

In [ ]:
X_lda = LDA.transform(X_train)
if X_lda.shape[1] >= 2:
    plt.xlabel('LD1')
    plt.ylabel('LD2')
    plt.scatter(
        X_lda[:,0],
        X_lda[:,1],
        c=y,
        cmap='rainbow',
        alpha=0.5,
        edgecolors='b'
    )
    plt.show(block=True)
print(LDA.explained_variance_ratio_)
print(X_lda[:5])

In [38]:
"""
repeated tests
"""
from tqdm import tqdm
test_num = 100
accuracies = []
for i_test in tqdm(range(test_num)):
    np.random.seed(int(time.time()))
    random_state = np.random.randint(0, 1e4)
    X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=0.30, random_state=random_state, shuffle=True, stratify=label)
    y_train = y_train.squeeze()
    LDA = LinearDiscriminantAnalysis()
    LDA.fit(X_train, y_train)
    predictions = LDA.predict(X_test)
    accuracy = precision_score(y_test, predictions)
    accuracies.append(accuracy)
accuracies = np.array(accuracies)
df_describe = pd.DataFrame(accuracies)
df_describe.describe()

100%|██████████| 100/100 [00:04<00:00, 22.19it/s]


,0
count,100.000000
mean,0.911066
std,0.036459
min,0.859155
25%,0.904762
50%,0.906250
75%,0.919355
max,0.966102


## repeated tests (many models)

In [93]:
from sklearn.model_selection import train_test_split,cross_val_score,RepeatedStratifiedKFold,GridSearchCV, RandomizedSearchCV

In [22]:
def fit_model(X_train, y_train, X_test, y_test, model, precisions, recalls, accuracies, one_hot_y):
    if one_hot_y:
        y_train = pd.get_dummies(y_train)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    if one_hot_y:
        predictions = np.array([np.argmax(i) for i in predictions])
    #conf_mat = confusion_matrix(y_test, predictions)
    
    precision = precision_score(y_test, predictions)
    recall = recall_score(y_test, predictions)
    precisions.append(precision)
    recalls.append(recall)

    if type(y_test) == np.ndarray:
        diffs = np.where(predictions!=y_test.squeeze())
    else:
        diffs = np.where(predictions!=y_test.values.squeeze())
    accuracy = 1-len(diffs[0])/len(y_test)
    accuracies.append(accuracy)
    return precisions, recalls, accuracies

def show_results(precisions, recalls, accuracies, log_path=None):
    precisions = np.array(precisions)
    df_describe = pd.DataFrame(precisions)
    log_xalg('precision:', log_path=log_path)
    log_xalg(df_describe.describe(), '\n', log_path=log_path)
    recalls = np.array(recalls)
    df_describe = pd.DataFrame(recalls)
    log_xalg('recall:', log_path=log_path)
    log_xalg(df_describe.describe(), '\n', log_path=log_path)

    accuracies = np.array(accuracies)
    df_describe = pd.DataFrame(accuracies)
    log_xalg('accuracy:', log_path=log_path)
    log_xalg(df_describe.describe(), log_path=log_path)

def repeated_tests(model_class, X, label, split_ratio=0.3, test_num=100, model_params=None, seed=None, log_path=None, one_hot_y=False):
    from tqdm import tqdm
    precisions = []
    recalls = []
    accuracies = []

    if type(model_class) is list:
        precisions = {}
        recalls = {}
        accuracies = {}

    for i_test in tqdm(range(test_num)):
        if seed is None:
            np.random.seed(int(time.time()))
        random_state = np.random.randint(0, 1e4)
        if type(model_class) is list:
            for i_model, one_model_class in enumerate(model_class):
                X_train, X_test, y_train, y_test = train_test_split(
                    X[i_model], label, test_size=split_ratio, random_state=random_state, shuffle=True, stratify=label)
                y_train = y_train.squeeze()
                    
                if model_params is None:
                    model = one_model_class()
                else:
                    assert type(model_params) is list
                    if model_params[i_model] is None:
                        model = one_model_class()
                    else:
                        params = dict(model_params[i_model])
                        model = one_model_class(**params)

                if i_model not in precisions:
                    precisions[i_model] = []
                    recalls[i_model] = []
                    accuracies[i_model] = []
                fit_model(X_train, y_train, X_test, y_test, model, precisions[i_model], recalls[i_model], accuracies[i_model], one_hot_y=one_hot_y)
        else:
            X_train, X_test, y_train, y_test = train_test_split(
                X, label, test_size=split_ratio, random_state=random_state, shuffle=True, stratify=label)
            y_train = y_train.squeeze()

            if model_params is None:
                model = model_class()
            else:
                model = model_class(**model_params)
            fit_model(X_train, y_train, X_test, y_test, model, precisions, recalls, accuracies, one_hot_y=one_hot_y)

    print('start show results')
    if type(model_class) is list:
        for i_model, model in enumerate(model_class):
            print('i_model:', i_model)
            show_results(precisions[i_model], recalls[i_model], accuracies[i_model], log_path)
    else:
        print(X.shape, X_train.shape, X_test.shape, '\n')
        show_results(precisions, recalls, accuracies, log_path)
    return precisions, recalls, accuracies

In [28]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np
#,'max_depth':[2,4,6,8,10]
model = DecisionTreeClassifier()
cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=2, random_state=0)
grid = dict()
grid['criterion'] = ['gini', 'entropy', 'log_loss']
#RandomizedSearchCV GridSearchCV
search = RandomizedSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=-1)
results = search.fit(X_norm_with_spatial_features_norm, label.values)
print('metric: %.3f' % results.best_score_)
print('model_params=',results.best_params_)

c:\Users\AFSALab\.conda\envs\pytorch\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning:

The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.



metric: 0.749
model_params= {'criterion': 'entropy'}


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
np.random.seed(0)
seed=True
#repeated_tests(DecisionTreeClassifier, X_norm, label, split_ratio=0.3, test_num=100, seed=seed)
#repeated_tests(DecisionTreeClassifier, spatial_features_norm, label, split_ratio=0.3, test_num=100, seed=seed)
#repeated_tests(DecisionTreeClassifier, X_norm_with_spatial_features_norm, label, split_ratio=0.3, test_num=100, seed=seed)

#repeated_tests(DecisionTreeClassifier, X_norm_with_spatial_features_norm_MRMR_norm, label, split_ratio=0.3, test_num=100, seed=seed)

#repeated_tests(DecisionTreeClassifier, X_norm_mrmr, label, split_ratio=0.3, test_num=100, seed=seed)
#repeated_tests(DecisionTreeClassifier, spatial_features_mrmr, label, split_ratio=0.3, test_num=100, seed=seed)
_=repeated_tests(DecisionTreeClassifier, X_norm_mrmr_with_spatial_features_norm_mrmr, label, split_ratio=0.3, test_num=100, seed=seed)

#repeated_tests(DecisionTreeClassifier, X_norm_with_spatial_features_norm_PCA_norm, label, split_ratio=0.3, test_num=100, seed=seed)

100%|██████████| 100/100 [00:02<00:00, 42.49it/s]

start show results
(420, 69) (294, 69) (126, 69) 

precision:
                0
count  100.000000
mean     0.760307
std      0.048162
min      0.650794
25%      0.729437
50%      0.762712
75%      0.790323
max      0.881356 

recall:
                0
count  100.000000
mean     0.756508
std      0.046523
min      0.650794
25%      0.730159
50%      0.753968
75%      0.793651
max      0.888889 

accuracy:
                0
count  100.000000
mean     0.757778
std      0.040570
min      0.650794
25%      0.730159
50%      0.753968
75%      0.785714
max      0.857143


In [37]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import numpy as np
#,'max_depth':[2,4,6,8,10]
model = RandomForestClassifier()
cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=2, random_state=0)
grid = dict()
grid['n_estimators'] = np.arange(10, 110, 10).tolist()
# grid['min_samples_leaf'] = [1,2,4]
# grid['max_features'] = ['sqrt', 'log2', None]
grid['criterion'] = ['gini', 'entropy', 'log_loss']
#RandomizedSearchCV GridSearchCV
search = GridSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=-1)
results = search.fit(X_norm_with_spatial_features_norm, label.values)
print('metric: %.3f' % results.best_score_)
print('model_params=',results.best_params_)

c:\Users\AFSALab\.conda\envs\pytorch\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



metric: 0.835
model_params= {'criterion': 'entropy', 'n_estimators': 90}


In [35]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_validate
np.random.seed(0)
seed=True
#model_params={'n_estimators': 100, 'min_samples_leaf': 1, 'max_features': None, 'criterion': 'gini'}
#_=repeated_tests(RandomForestClassifier, X_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)
#model_params= {'n_estimators': 30, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'criterion': 'log_loss'}
#_=repeated_tests(RandomForestClassifier, spatial_features_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

#model_params= {'n_estimators': 30, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'criterion': 'entropy'}
#repeated_tests(RandomForestClassifier, X_norm_with_spatial_features_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

#model_params={'criterion': 'gini', 'max_features': None, 'min_samples_leaf': 1, 'n_estimators': 30}
#precisions, recalls, accuracies = repeated_tests(RandomForestClassifier, X_norm_with_spatial_features_norm_MRMR_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)
model_params= {'n_estimators': 60, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'criterion': 'entropy'}
precisions, recalls, accuracies = repeated_tests(RandomForestClassifier, X_norm_mrmr_with_spatial_features_norm_mrmr, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

#model_params= {'n_estimators': 90, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'criterion': 'gini'}
#precisions, recalls, accuracies = repeated_tests(RandomForestClassifier, X_norm_with_spatial_features_norm_PCA_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

100%|██████████| 100/100 [00:16<00:00,  5.91it/s]

start show results
(420, 69) (294, 69) (126, 69) 

precision:
                0
count  100.000000
mean     0.859404
std      0.038785
min      0.774194
25%      0.828125
50%      0.862069
75%      0.891396
max      0.959184 

recall:
                0
count  100.000000
mean     0.790476
std      0.046291
min      0.650794
25%      0.761905
50%      0.793651
75%      0.825397
max      0.888889 

accuracy:
                0
count  100.000000
mean     0.829762
std      0.029853
min      0.761905
25%      0.809524
50%      0.833333
75%      0.849206
max      0.896825


In [59]:
from sklearn.linear_model import LogisticRegression
import numpy as np
#,'max_depth':[2,4,6,8,10]
model = LogisticRegression()
cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=2, random_state=0)
grid = dict()
grid['solver'] = ['lbfgs', 'liblinear',  'newton-cg', 'newton-cholesky', 'sag', 'saga']
# grid['penalty'] = ["l1", "l2", "elasticnet", None]

grid['solver'] = ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag', ]
grid['penalty'] = ["l2", None]

# grid['solver'] = ['liblinear', ]
# grid['penalty'] = ["l2", 'l1']

# grid['solver'] = ['saga', ]
# grid['penalty'] = ["l1", "l2", "elasticnet", None]

#RandomizedSearchCV GridSearchCV
search = GridSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=-1)
results = search.fit(X_norm_with_spatial_features_norm, label.values)
print('metric: %.3f' % results.best_score_)
print('model_params=',results.best_params_)

c:\Users\AFSALab\.conda\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



metric: 0.871
model_params= {'penalty': 'l2', 'solver': 'lbfgs'}


c:\Users\AFSALab\.conda\envs\pytorch\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [ ]:
from sklearn.linear_model import LogisticRegression
#solver = lbfgs liblinear
np.random.seed(0)
seed=True
model_params={'solver': 'liblinear'}
#repeated_tests(LogisticRegression, X_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)
#repeated_tests(LogisticRegression, spatial_features_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)
#_=repeated_tests(LogisticRegression, X_norm_with_spatial_features_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

#repeated_tests(LogisticRegression, X_norm_with_spatial_features_norm_MRMR_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)
model_params= {'solver': 'saga'}
_=repeated_tests(LogisticRegression, X_norm_mrmr_with_spatial_features_norm_mrmr, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

#repeated_tests(LogisticRegression, X_norm_with_spatial_features_norm_PCA_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

In [ ]:
from sklearn.svm import LinearSVC
import numpy as np
model = LinearSVC()
cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=2, random_state=0)
grid = dict()
#grid['C'] = [1,5, 10, 50, 100, 500, 1000]
grid['C'] = np.linspace(0.0, 5, 20)
#grid['gamma'] = [1,0.1,0.001,0.0001]
#grid['kernel'] = ['linear','rbf']
#RandomizedSearchCV GridSearchCV
search = GridSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=-1)
# X_norm
# spatial_features_norm
# X_norm_with_spatial_features_norm
results = search.fit(X_norm, label.values)
print('metric: %.3f' % results.best_score_)
print('model_params=',results.best_params_)

In [ ]:
from sklearn.svm import LinearSVC
np.random.seed(0)
seed=True
#C=1.0

#model_params={'C': 5}
#_=repeated_tests(LinearSVC, X_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

#model_params= {'C': 1}
#_=repeated_tests(LinearSVC, spatial_features_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

#model_params= {'C': 1}
#_=repeated_tests(LinearSVC, X_norm_with_spatial_features_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

model_params= {'C': 1}
#_=repeated_tests(LinearSVC, X_norm_with_spatial_features_norm_MRMR_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)
_=repeated_tests(LinearSVC, X_norm_mrmr_with_spatial_features_norm_mrmr, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

#model_params= {'C': 1}
#_=repeated_tests(LinearSVC, X_norm_with_spatial_features_norm_PCA_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

In [319]:
from xgboost import XGBRegressor, XGBClassifier
import numpy as np
#,'max_depth':[2,4,6,8,10]
model = XGBClassifier()
cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=2, random_state=0)
grid = dict()
grid['n_estimators'] = np.arange(50, 500, 50).tolist()
grid['min_child_weight'] = [1, 5, 10]
grid['gamma'] = [0.5, 1, 1.5, 2, 5]
grid['subsample'] = [0.6, 0.8, 1.0]
grid['colsample_bytree'] = [0.6, 0.8, 1.0]
grid['max_depth'] = [3, 4, 5]
#RandomizedSearchCV GridSearchCV
search = RandomizedSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=-1)
results = search.fit(X_norm_mrmr_with_spatial_features_norm_mrmr, label.values)
print('metric: %.3f' % results.best_score_)
print('model_params=',results.best_params_)

metric: 0.844
model_params= {'subsample': 1.0, 'n_estimators': 100, 'min_child_weight': 1, 'max_depth': 4, 'gamma': 0.5, 'colsample_bytree': 0.6}


In [ ]:
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from xgboost import XGBRegressor, XGBClassifier
#from lightgbm import LGBMRegressor, LGBMClassifier

np.random.seed(0)
seed=True
#model_params={'n_estimators':400, 'learning_rate': 0.1}
#model_params=None

#model_params={'subsample': 0.6, 'n_estimators': 150, 'min_child_weight': 1, 'max_depth': 4, 'gamma': 0.5, 'colsample_bytree': 1.0}
#repeated_tests(XGBClassifier, X_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

#model_params= {'subsample': 0.8, 'n_estimators': 450, 'min_child_weight': 10, 'max_depth': 4, 'gamma': 0.5, 'colsample_bytree': 0.8}
#repeated_tests(XGBClassifier, spatial_features_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

#model_params= {'subsample': 1.0, 'n_estimators': 250, 'min_child_weight': 10, 'max_depth': 4, 'gamma': 1, 'colsample_bytree': 1.0}
#repeated_tests(XGBClassifier, X_norm_with_spatial_features_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

model_params= {'subsample': 1.0, 'n_estimators': 100, 'min_child_weight': 1, 'max_depth': 4, 'gamma': 0.5, 'colsample_bytree': 0.6}
#repeated_tests(XGBClassifier, X_norm_with_spatial_features_norm_MRMR_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)
repeated_tests(XGBClassifier, X_norm_mrmr_with_spatial_features_norm_mrmr, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

#model_params={'subsample': 0.6, 'n_estimators': 100, 'min_child_weight': 1, 'max_depth': 3, 'gamma': 1, 'colsample_bytree': 0.8}
#precisions, recalls, accuracies =repeated_tests(XGBClassifier, X_norm_with_spatial_features_norm_PCA_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

In [ ]:
"""
section PLS-DA
"""

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
np.random.seed(0)
seed=True
#_=repeated_tests(LinearDiscriminantAnalysis, X_norm, label, split_ratio=0.3, test_num=100, seed=seed)

#_=repeated_tests(LinearDiscriminantAnalysis, spatial_features_norm, label, split_ratio=0.3, test_num=100, seed=seed)
#_=repeated_tests(LinearDiscriminantAnalysis, X_norm_with_spatial_features_norm_no_bsif, label, split_ratio=0.3, test_num=100, seed=seed)
#_=repeated_tests(LinearDiscriminantAnalysis, bsif_features_norm, label, split_ratio=0.3, test_num=100, seed=seed)

#_=repeated_tests(LinearDiscriminantAnalysis, X_norm_with_spatial_features_norm, label, split_ratio=0.3, test_num=100, seed=seed)
#_=repeated_tests(LinearDiscriminantAnalysis, X_norm_with_spatial_features_norm_hara_lbp_hu, label, split_ratio=0.3, test_num=100, seed=seed)

#_=repeated_tests(LinearDiscriminantAnalysis, X_norm_with_spatial_features_norm_MRMR_norm, label, split_ratio=0.3, test_num=100, seed=seed)
_=repeated_tests(LinearDiscriminantAnalysis, X_norm_mrmr_with_spatial_features_norm_mrmr, label, split_ratio=0.3, test_num=100, seed=seed)

#_=repeated_tests(LinearDiscriminantAnalysis, X_norm_with_spatial_features_norm_PCA_norm, label, split_ratio=0.3, test_num=100, seed=seed)

In [44]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
np.random.seed(0)
seed=True
#_=repeated_tests(QuadraticDiscriminantAnalysis, X_norm, label, split_ratio=0.3, test_num=100, seed=seed)
#_=repeated_tests(QuadraticDiscriminantAnalysis, spatial_features_norm, label, split_ratio=0.3, test_num=100, seed=seed)
#_=repeated_tests(QuadraticDiscriminantAnalysis, X_norm_with_spatial_features_norm, label, split_ratio=0.3, test_num=100, seed=seed)
#_=repeated_tests(QuadraticDiscriminantAnalysis, X_norm_with_spatial_features_norm_MRMR_norm, label, split_ratio=0.3, test_num=100, seed=seed)
_=repeated_tests(QuadraticDiscriminantAnalysis, X_norm_mrmr_with_spatial_features_norm_mrmr, label, split_ratio=0.3, test_num=100, seed=seed)
#_=repeated_tests(QuadraticDiscriminantAnalysis, X_norm_with_spatial_features_norm_PCA_norm, label, split_ratio=0.3, test_num=100, seed=seed)

100%|██████████| 100/100 [00:05<00:00, 18.75it/s]

start show results
(420, 69) (294, 69) (126, 69) 

precision:
                0
count  100.000000
mean     0.797211
std      0.042206
min      0.698630
25%      0.771429
50%      0.794118
75%      0.826462
max      0.919355 

recall:
                0
count  100.000000
mean     0.858254
std      0.049647
min      0.730159
25%      0.825397
50%      0.857143
75%      0.904762
max      0.968254 

accuracy:
                0
count  100.000000
mean     0.818175
std      0.031120
min      0.730159
25%      0.799603
50%      0.825397
75%      0.841270
max      0.912698


## SMOTE / GridSearchCV

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score,RepeatedStratifiedKFold,GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import ConfusionMatrixDisplay,precision_score,recall_score,confusion_matrix
from imblearn.over_sampling import SMOTE

In [146]:
oversample = SMOTE()
X_smote, y_smote = oversample.fit_resample(X, y)
random_state = 42
Xs_train, Xs_test, ys_train, ys_test = train_test_split(X_smote, y_smote, test_size=0.30, random_state=random_state, shuffle=True, stratify=y_smote)
X.shape, X_smote.shape, Xs_train.shape, Xs_test.shape

((210, 462), (252, 462), (176, 462), (76, 462))

In [ ]:
X_train, X_test, y_train, y_test
oversample = SMOTE()
Xs_train, ys_train = oversample.fit_resample(X_train, y_train)
Xs_test, ys_test = X_test, y_test
X.shape, X_train.shape, Xs_train.shape, Xs_test.shape

## repeated_tests_with_SMOTE

In [331]:
def repeated_tests_with_SMOTE(model_class, X, label, split_ratio=0.3, test_num=30, model_params=None):
    from tqdm import tqdm
    accuracies = []
    for i_test in tqdm(range(test_num)):
        np.random.seed(int(time.time()))
        random_state = np.random.randint(0, 1e4)
        X_train, X_test, y_train, y_test = train_test_split(
            X, label, test_size=split_ratio, random_state=random_state, shuffle=True, stratify=label)
        oversample = SMOTE()
        Xs_train, ys_train = oversample.fit_resample(X_train, y_train)

        ys_train = ys_train.squeeze()
        if model_params is None:
            model = model_class()
        else:
            model = model_class(**model_params)

        model.fit(Xs_train, ys_train)
        predictions = model.predict(X_test)
        precision = precision_score(y_test, predictions)
        recall = recall_score(y_test, predictions)
        precisions.append(precision)
        recalls.append(recall)

        diffs = np.where(predictions!=y_test.values.squeeze())
        accuracy = 1-len(diffs[0])/len(y_test)
        accuracies.append(accuracy)

    print(X.shape, X_train.shape, X_test.shape)
    precisions = np.array(precisions)
    df_describe = pd.DataFrame(precisions)
    print(df_describe.describe())
    recalls = np.array(recalls)
    df_describe = pd.DataFrame(recalls)
    print(df_describe.describe())

    accuracies = np.array(accuracies)
    df_describe = pd.DataFrame(accuracies)
    print(df_describe.describe())

## rLDA

In [236]:
# X
# X_norm
# spatial_features_norm
# X_norm_with_spatial_features_norm
X_train, X_test, y_train, y_test = train_test_split(X_norm_with_spatial_features_norm, label, test_size=0.30, random_state=42, shuffle=True, stratify=label)
oversample = SMOTE()
Xs_train, ys_train = oversample.fit_resample(X_train, y_train)
Xs_test, ys_test = X_test, y_test
X.shape, X_train.shape, Xs_train.shape, Xs_test.shape, y_test.shape

((420, 462), (294, 2964), (294, 2964), (126, 2964), (126, 1))

In [84]:
import numpy as np

LDA = LinearDiscriminantAnalysis()

cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=2, random_state=0)

grid = dict()
#grid['solver'] = ['eigen','lsqr', None]
# LinearDiscriminantAnalysis
# shrinkages = np.linspace(0.000001, 0.00001, 10).tolist()

grid['tol'] = np.linspace(1e-5, 1e-3, 21).tolist()

# grid['shrinkage'] = ['auto'] + shrinkages
#RandomizedSearchCV GridSearchCV
search = GridSearchCV(LDA, grid, scoring='accuracy', cv=cv, n_jobs=-1)
#search = GridSearchCV(LDA, grid, scoring='precision', cv=cv, n_jobs=-1)
#results = search.fit(Xs_train, ys_train)

# X_norm
# spatial_features_norm
# X_norm_with_spatial_features_norm
results = search.fit(X_norm_with_spatial_features_norm, label.values)
print('metric: %.3f' % results.best_score_)
print('model_params=',results.best_params_)

c:\Users\AFSALab\.conda\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



metric: 0.845
model_params= {'tol': 1e-05}


In [91]:

import numpy as np
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

QDA = QuadraticDiscriminantAnalysis()

cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=2, random_state=0)

grid = dict()
#grid['solver'] = ['eigen','lsqr', None]
# LinearDiscriminantAnalysis
# shrinkages = np.linspace(0.000001, 0.00001, 10).tolist()

grid['tol'] = np.linspace(1e-6, 1e-5, 21).tolist()

# grid['shrinkage'] = ['auto'] + shrinkages
#RandomizedSearchCV GridSearchCV
search = GridSearchCV(QDA, grid, scoring='accuracy', cv=cv, n_jobs=-1)

# X_norm
# spatial_features_norm
# X_norm_with_spatial_features_norm
results = search.fit(spatial_features_norm, label.values)
print('metric: %.3f' % results.best_score_)
print('model_params=',results.best_params_)

c:\Users\AFSALab\.conda\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\Users\AFSALab\.conda\envs\pytorch\lib\site-packages\sklearn\discriminant_analysis.py:926: UserWarning:

Variables are collinear



metric: 0.492
model_params= {'tol': 1e-06}


In [ ]:
LDA_final=LinearDiscriminantAnalysis(shrinkage='auto', solver='eigen')
LDA_final.fit_transform(Xs_train, ys_train)
Xs_test_predictions=LDA_final.predict(Xs_test)

#ConfusionMatrixDisplay.from_predictions(y_test, X_test_predictions)
ConfusionMatrixDisplay.from_predictions(ys_test, Xs_test_predictions)
plt.show()

if type(y_test)==pd.DataFrame:
    tn, fp, fn, tp = confusion_matrix(list(y_test.values), list(Xs_test_predictions), labels=[0, 1]).ravel()
else:
    tn, fp, fn, tp = confusion_matrix(list(ys_test), list(Xs_test_predictions), labels=[0, 1]).ravel()

print('True Positive :', tp)
print('True Negative :', tn)
print('False Positive :', fp)
print('False Negative :', fn)
if type(y_test)==pd.DataFrame:
    print("Precision score",np.round(precision_score(y_test.values, Xs_test_predictions),3))
else:
    print("Precision score",np.round(precision_score(ys_test, Xs_test_predictions),3))

In [ ]:
model_params ={'shrinkage':'auto', 'solver':'eigen'}
repeated_tests(LinearDiscriminantAnalysis, X, label, split_ratio=0.3, test_num=100, model_params=model_params)

In [ ]:
np.random.seed(0)
seed=True
#model_params ={'shrinkage': 5e-06, 'solver':'eigen'}
##repeated_tests(LinearDiscriminantAnalysis, X, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)
#repeated_tests(LinearDiscriminantAnalysis, X_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

#model_params ={'shrinkage': 'auto', 'solver':'eigen'}
#repeated_tests(LinearDiscriminantAnalysis, spatial_features_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

#model_params ={'shrinkage': 'auto', 'solver':'eigen'}
#repeated_tests(LinearDiscriminantAnalysis, X_norm_with_spatial_features_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

#model_params ={'solver': 'eigen', 'shrinkage': 'auto'}
#_=repeated_tests(LinearDiscriminantAnalysis, X_norm_with_spatial_features_norm_MRMR_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)
model_params ={'solver': 'eigen', 'shrinkage': 1e-06}
_=repeated_tests(LinearDiscriminantAnalysis, X_norm_mrmr_with_spatial_features_norm_mrmr, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

#model_params ={'shrinkage': 'auto', 'solver':'eigen'}
#_=repeated_tests(LinearDiscriminantAnalysis, X_norm_with_spatial_features_norm_PCA_norm, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

#model_params= {'solver': 'eigen', 'shrinkage': 3e-06}
#_=repeated_tests(LinearDiscriminantAnalysis, X_norm_mrmr, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)
#model_params= {'solver': 'eigen', 'shrinkage': 'auto'}
#_=repeated_tests(LinearDiscriminantAnalysis, spatial_features_norm_MRMR, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params)

In [ ]:
model_params ={'shrinkage':'auto', 'solver':'eigen'}
repeated_tests_with_SMOTE(LinearDiscriminantAnalysis, X, label, split_ratio=0.3, test_num=100, model_params=model_params)

In [ ]:
repeated_tests_with_SMOTE(LinearDiscriminantAnalysis, X, label, split_ratio=0.3, test_num=100, model_params=None)

## rLDA + spatial features (load data)

In [10]:
from sklearn.model_selection import train_test_split,cross_val_score,RepeatedStratifiedKFold,GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import ConfusionMatrixDisplay,accuracy_score,precision_score,recall_score,confusion_matrix
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import time

In [11]:
from modeling_spectral_data import load_im_dirs
im_dirs = load_im_dirs()
len(im_dirs)

from modeling_spectral_data import load_spectral_mean_data
hyper_dict_good_tot, hyper_dict_bad_tot = load_spectral_mean_data(use_part_1=1, use_part_2=1)
len(hyper_dict_good_tot)

from modeling_spectral_data import spli_mean_spectral_data
hyper_dict_good_tot_0, hyper_dict_good_tot_1, hyper_dict_bad_tot_0, hyper_dict_bad_tot_1 = spli_mean_spectral_data(hyper_dict_good_tot, hyper_dict_bad_tot)
len(hyper_dict_good_tot_0)

84

In [12]:
#band_list = [600, 750, 900]
#band_list = [812.8,
# 808.75,
# 811.44,
# 413.52,
# 410.88,
# 412.2,
# 712.06,
# 713.4,
# 710.72,
# 803.36,
# 804.71,
# 800.68,
# 690.66,
# 692.0,
# 689.32,
# 625.26,
# 622.6,
# 623.93,
# 635.92,
# 634.59,
# 633.26]

# good
band_list = [413.52, 599.97, 712.06, 736.18, 812.8, 911.41]
# worsen
#band_list = [599.97, 736.18, 911.41]
#band_list = [413.52, 599.97, 712.06, 812.8, 911.41]

In [ ]:
"""
generate data
"""

from modeling_CNN_blueberry import HyperData
from tqdm import tqdm

good_dataset = HyperData(save_dir_im_data_good, band_list, 0, hyper_dict_good_tot_0, wavelengths)
bad_dataset = HyperData(save_dir_im_data_bad, band_list, 1, hyper_dict_bad_tot_0, wavelengths)
good_dataset1 = HyperData(save_dir_im_data_good1, band_list, 0, hyper_dict_good_tot_1, wavelengths)
bad_dataset1 = HyperData(save_dir_im_data_bad1, band_list, 1, hyper_dict_bad_tot_1, wavelengths)

all_spatial_features = []
all_mean_spectral_features = []
for i_sample, im_dir in tqdm(enumerate(im_dirs)):
    if i_sample < len(good_dataset):
        dataset = good_dataset
        sample_idx = i_sample
    elif i_sample < len(good_dataset) + len(good_dataset1):
        dataset = good_dataset1
        sample_idx = i_sample - len(good_dataset)
    elif i_sample < len(good_dataset) + len(good_dataset1) + len(bad_dataset):
        dataset = bad_dataset
        sample_idx = i_sample -len(good_dataset) - len(good_dataset1)
    else:
        dataset = bad_dataset1
        sample_idx = i_sample - len(good_dataset) - len(good_dataset1) - len(bad_dataset)
    imgs, labels, spectral_features, spatial_features, sample_dir, spatial_features_num_dict = dataset.__getitem__(sample_idx)
    all_spatial_features.append(spatial_features)
    all_mean_spectral_features.append(spectral_features)
all_spatial_features = np.array(all_spatial_features)
all_mean_spectral_features = np.array(all_mean_spectral_features)

In [ ]:
print(all_spatial_features.shape)
print(spatial_features_num_dict)
print('feature number per waveband:')
for key in spatial_features_num_dict:
    if key != 'order':
        print(key, spatial_features_num_dict[key]/len(band_list))

In [13]:
#all_spatial_features all_spatial_features_21bands all_spatial_features_3bands all_spatial_features_5bands
save_path_all_spatial_features = r'D:\Dataset\HyperspectralBlueberry\mean_data_python\all_spatial_features_6bands.npy'

In [18]:
#np.save(save_path_all_spatial_features, all_spatial_features)

In [14]:
all_spatial_features = np.load(save_path_all_spatial_features, allow_pickle='TRUE')
all_spatial_features.shape

(420, 2502)

In [15]:
spatial_feature_means = all_spatial_features.mean(axis=0)
spatial_feature_stds = all_spatial_features.std(axis=0)

"""
spatial feature norm
"""
all_spatial_features_norm = (all_spatial_features - spatial_feature_means) / spatial_feature_stds
all_spatial_features.mean(0)[:10], all_spatial_features.std(0)[:10], all_spatial_features_norm.mean(0)[:10], all_spatial_features_norm.std(0)[:10]

(array([  0.1229,   0.545 , 698.3388,   0.545 ,   0.7853,   5.3088,
          3.1833,   2.1255,   2.5266,   0.0533]),
 array([  0.0634,   0.1455, 541.0698,   0.1455,   0.0425,   1.0179,
          0.7727,   0.2713,   0.3602,   0.0078]),
 array([ 0., -0., -0., -0.,  0., -0., -0., -0., -0., -0.]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))

In [16]:
from modeling_spectral_data import load_spectral_mean_data_xy
x, y = load_spectral_mean_data_xy(use_part_1=1, use_part_2=1)
X =  pd.DataFrame(x)
label = pd.DataFrame(y, columns=['label'])
#label = label.values.ravel()

X = x
X_norm = (x-x.mean(0))/x.std(0)

X.shape, label.shape, x.mean(0)[:5], x.std(0)[:5], X_norm.mean(0)[:5], X_norm.std(0)[:5]

((420, 462),
 (420, 1),
 array([0.1516, 0.1501, 0.1481, 0.1467, 0.1463]),
 array([0.0411, 0.0426, 0.0439, 0.0451, 0.0462]),
 array([-0., -0., -0., -0.,  0.]),
 array([1., 1., 1., 1., 1.]))

In [17]:
#spatial_features_num_dict={'haralick_features': 39,
# 'lbp_features': 78,
# 'hu_moments_features': 21,
# 'gabor_features': 300,
# 'bsif_features': 300,
# 'order': ['haralick_features',
#  'lbp_features',
#  'hu_moments_features',
#  'gabor_features',
#  'bsif_features']}

#spatial_features_num_dict={'haralick_features': 273,
# 'lbp_features': 546,
# 'hu_moments_features': 147,
# 'gabor_features': 2100,
# 'bsif_features': 2100,
# 'order': ['haralick_features',
#  'lbp_features',
#  'hu_moments_features',
#  'gabor_features',
#  'bsif_features']}

#spatial_features_num_dict={'haralick_features': 78,
# 'lbp_features': 156,
# 'hu_moments_features': 42,
# 'gabor_features': 600,
# 'bsif_features': 600,
# 'order': ['haralick_features',
#  'lbp_features',
#  'hu_moments_features',
#  'gabor_features',
#  'bsif_features']}

spatial_features_num_dict={
    'haralick_features': 168, 'lbp_features': 354, 'hu_moments_features': 42, 'gabor_features': 402, 'bsif_features': 1536,
    'order': ['haralick_features', 'lbp_features', 'hu_moments_features', 'gabor_features', 'bsif_features']
    }

In [18]:
"""
average
"""
im_num_each_sample = len(band_list)
haralick_num = spatial_features_num_dict['haralick_features']
lbp_num = spatial_features_num_dict['lbp_features']
hu_num = spatial_features_num_dict['hu_moments_features']
gabor_num = spatial_features_num_dict['gabor_features']
bsif_num = spatial_features_num_dict['bsif_features']
# haralick_num + lbp_num + hu_num + gabor_num + bsif_num
start = 0

spatial_features_order = spatial_features_num_dict['order']

haralick_features = all_spatial_features[:, start: start+haralick_num]
haralick_features_mean = haralick_features.reshape(-1, haralick_num//im_num_each_sample, im_num_each_sample).mean(2)

lbp_features = all_spatial_features[:, start+haralick_num: start+haralick_num+lbp_num]
lbp_features_mean = lbp_features.reshape(-1, lbp_num//im_num_each_sample, im_num_each_sample).mean(2)

hu_moments_features = all_spatial_features[:, start+haralick_num+lbp_num: start+haralick_num+lbp_num+hu_num]
hu_moments_features_mean = hu_moments_features.reshape(-1, hu_num//im_num_each_sample, im_num_each_sample).mean(2)

gabor_features = all_spatial_features[:, start+haralick_num+lbp_num+hu_num: start+haralick_num+lbp_num+hu_num+gabor_num]
gabor_features_mean = gabor_features.reshape(-1, gabor_num//im_num_each_sample, im_num_each_sample).mean(2)

bsif_features = all_spatial_features[:, start+haralick_num+lbp_num+hu_num+gabor_num: start+haralick_num+lbp_num+hu_num+gabor_num+bsif_num]
bsif_features_mean = bsif_features.reshape(-1, bsif_num//im_num_each_sample, im_num_each_sample).mean(2)

haralick_features_norm =  (haralick_features - haralick_features.mean(axis=0))/haralick_features.std(axis=0)
lbp_features_norm = (lbp_features - lbp_features.mean(axis=0))/lbp_features.std(axis=0)
hu_moments_features_norm =  (hu_moments_features - hu_moments_features.mean(axis=0))/hu_moments_features.std(axis=0)
gabor_features_norm =  (gabor_features - gabor_features.mean(axis=0))/gabor_features.std(axis=0)
bsif_features_norm =  (bsif_features - bsif_features.mean(axis=0))/bsif_features.std(axis=0)

haralick_features_norm_mean =  (haralick_features_mean - haralick_features_mean.mean(axis=0))/haralick_features_mean.std(axis=0)
lbp_features_norm_mean = (lbp_features_mean - lbp_features_mean.mean(axis=0))/lbp_features_mean.std(axis=0)
hu_moments_features_norm_mean =  (hu_moments_features_mean - hu_moments_features_mean.mean(axis=0))/hu_moments_features_mean.std(axis=0)
gabor_features_norm_mean =  (gabor_features_mean - gabor_features_mean.mean(axis=0))/gabor_features_mean.std(axis=0)
bsif_features_norm_mean =  (bsif_features_mean - bsif_features_mean.mean(axis=0))/bsif_features_mean.std(axis=0)

# best combination? [haralick_features, lbp_features, hu_moments_features]
spatial_features = np.concatenate([haralick_features, lbp_features, hu_moments_features, gabor_features, bsif_features], axis=1)
spatial_features_norm = np.concatenate([haralick_features_norm, lbp_features_norm, hu_moments_features_norm, gabor_features_norm, bsif_features_norm], axis=1)
spatial_features_norm_no_bsif = np.concatenate([haralick_features_norm, lbp_features_norm, hu_moments_features_norm, gabor_features_norm], axis=1)

spatial_features_mean = np.concatenate([haralick_features_mean, lbp_features_mean, hu_moments_features_mean, gabor_features_mean, bsif_features_mean], axis=1)
spatial_features_norm_mean = np.concatenate([haralick_features_norm_mean, lbp_features_norm_mean, hu_moments_features_norm_mean, gabor_features_norm_mean, bsif_features_norm_mean], axis=1)
spatial_features_norm.shape

(420, 2502)

In [18]:
X_with_spatial_features = np.concatenate([X, spatial_features], axis=1)
X_norm_with_spatial_features_norm = np.concatenate([X_norm, spatial_features_norm], axis=1)
X_norm_with_spatial_features_norm_no_bsif = np.concatenate([X_norm, spatial_features_norm_no_bsif], axis=1)
X_norm_with_spatial_features_norm.shape

(420, 2964)

In [25]:
# import scipy.io
# scipy.io.savemat(r'D:\BoyangDeng\BlueberryClassification\datasets\mean_data_python\label.mat', {'label': np.squeeze(label.values)})
# scipy.io.savemat(r'D:\BoyangDeng\BlueberryClassification\datasets\mean_data_python\spectral_features.mat', {'X_norm': X_norm, 'X': X})
# scipy.io.savemat(r'D:\BoyangDeng\BlueberryClassification\datasets\mean_data_python\spatial_features_norm_6bands.mat', {'spatial_features_norm': spatial_features_norm, 'spatial_features': spatial_features})
# scipy.io.savemat(r'D:\BoyangDeng\BlueberryClassification\datasets\mean_data_python\spectral_features_with_spatial_features_norm_6bands.mat', {'X_norm_with_spatial_features_norm': X_norm_with_spatial_features_norm, 'X_with_spatial_features': X_with_spatial_features})

## MRMR
relative to feature extraction -> MRMR section

In [28]:
import scipy.io
spectral_features_mrmr_idxes=scipy.io.loadmat(r'D:\BoyangDeng\test\spectral_features_mrmr_idxes.mat')
spectral_features_mrmr_scores=scipy.io.loadmat(r'D:\BoyangDeng\test\spectral_features_mrmr_scores.mat')
spectral_features_mrmr_idxes=spectral_features_mrmr_idxes['idx'][0]
spectral_features_mrmr_scores=spectral_features_mrmr_scores['score'][0]

haralick_features_mrmr_idxes=scipy.io.loadmat(r'D:\BoyangDeng\test\haralick_features_mrmr_idxes.mat')
haralick_features_mrmr_scores=scipy.io.loadmat(r'D:\BoyangDeng\test\haralick_features_mrmr_scores.mat')
haralick_features_mrmr_idxes=haralick_features_mrmr_idxes['idx_1'][0]
haralick_features_mrmr_scores=haralick_features_mrmr_scores['score_1'][0]

lbp_features_mrmr_idxes=scipy.io.loadmat(r'D:\BoyangDeng\test\lbp_features_mrmr_idxes.mat')
lbp_features_mrmr_scores=scipy.io.loadmat(r'D:\BoyangDeng\test\lbp_features_mrmr_scores.mat')
lbp_features_mrmr_idxes=lbp_features_mrmr_idxes['idx_2'][0]
lbp_features_mrmr_scores=lbp_features_mrmr_scores['score_2'][0]

hu_features_mrmr_idxes=scipy.io.loadmat(r'D:\BoyangDeng\test\hu_features_mrmr_idxes.mat')
hu_features_mrmr_scores=scipy.io.loadmat(r'D:\BoyangDeng\test\hu_features_mrmr_scores.mat')
hu_features_mrmr_idxes=hu_features_mrmr_idxes['idx_3'][0]
hu_features_mrmr_scores=hu_features_mrmr_scores['score_3'][0]

gabor_features_mrmr_idxes=scipy.io.loadmat(r'D:\BoyangDeng\test\gabor_features_mrmr_idxes.mat')
gabor_features_mrmr_scores=scipy.io.loadmat(r'D:\BoyangDeng\test\gabor_features_mrmr_scores.mat')
gabor_features_mrmr_idxes=gabor_features_mrmr_idxes['idx_4'][0]
gabor_features_mrmr_scores=gabor_features_mrmr_scores['score_4'][0]

bsif_features_mrmr_idxes=scipy.io.loadmat(r'D:\BoyangDeng\test\bsif_features_mrmr_idxes.mat')
bsif_features_mrmr_scores=scipy.io.loadmat(r'D:\BoyangDeng\test\bsif_features_mrmr_scores.mat')
bsif_features_mrmr_idxes=bsif_features_mrmr_idxes['idx_5'][0]
bsif_features_mrmr_scores=bsif_features_mrmr_scores['score_5'][0]

In [27]:
show_statistics(bsif_features_mrmr_scores)

                 0
count  1536.000000
mean      0.013512
std       0.017470
min       0.000000
25%       0.003486
50%       0.008422
75%       0.016159
max       0.158046


In [29]:
import numpy as np
score_t=0.05
#score_t=0.005
valid_spectral_features_mrmr_idxes = spectral_features_mrmr_idxes[:np.sum(spectral_features_mrmr_scores>score_t)]-1

valid_haralick_features_mrmr_idxes = haralick_features_mrmr_idxes[:np.sum(haralick_features_mrmr_scores>score_t)]-1
valid_lbp_features_mrmr_idxes = lbp_features_mrmr_idxes[:np.sum(lbp_features_mrmr_scores>score_t)]-1
valid_hu_features_mrmr_idxes = hu_features_mrmr_idxes[:np.sum(hu_features_mrmr_scores>score_t)]-1
valid_gabor_features_mrmr_idxes = gabor_features_mrmr_idxes[:np.sum(gabor_features_mrmr_scores>score_t)]-1

score_t=0.05
#score_t=0.1
valid_bsif_features_mrmr_idxes = bsif_features_mrmr_idxes[:np.sum(bsif_features_mrmr_scores>score_t)]-1
print(valid_spectral_features_mrmr_idxes.shape)
print(valid_haralick_features_mrmr_idxes.shape)
print(valid_lbp_features_mrmr_idxes.shape)
print(valid_hu_features_mrmr_idxes.shape)
print(valid_gabor_features_mrmr_idxes.shape)
print(valid_bsif_features_mrmr_idxes.shape)

(33,)
(5,)
(3,)
(1,)
(1,)
(26,)


In [31]:
X_mrmr = X[:, valid_spectral_features_mrmr_idxes]

haralick_features_mrmr = haralick_features[:, valid_haralick_features_mrmr_idxes]
lbp_features_mrmr = lbp_features[:, valid_lbp_features_mrmr_idxes]
hu_moments_features_mrmr = hu_moments_features[:, valid_hu_features_mrmr_idxes]
gabor_features_mrmr = gabor_features[:, valid_gabor_features_mrmr_idxes]
bsif_features_mrmr = bsif_features[:, valid_bsif_features_mrmr_idxes]

X_norm_mrmr = X_norm[:, valid_spectral_features_mrmr_idxes]

haralick_features_norm_mrmr = haralick_features_norm[:, valid_haralick_features_mrmr_idxes]
lbp_features_norm_mrmr = lbp_features_norm[:, valid_lbp_features_mrmr_idxes]
hu_moments_features_norm_mrmr = hu_moments_features_norm[:, valid_hu_features_mrmr_idxes]
gabor_features_norm_mrmr = gabor_features_norm[:, valid_gabor_features_mrmr_idxes]
bsif_features_norm_mrmr = bsif_features_norm[:, valid_bsif_features_mrmr_idxes]

spatial_features_mrmr = np.concatenate([haralick_features_mrmr, lbp_features_mrmr, hu_moments_features_mrmr, gabor_features_mrmr, bsif_features_mrmr], axis=1)
spatial_features_norm_mrmr = np.concatenate([haralick_features_norm_mrmr, lbp_features_norm_mrmr, hu_moments_features_norm_mrmr, gabor_features_norm_mrmr, bsif_features_norm_mrmr], axis=1)

X_mrmr_with_spatial_features_mrmr = np.concatenate([X_mrmr, spatial_features_mrmr], axis=1)
X_norm_mrmr_with_spatial_features_norm_mrmr = np.concatenate([X_norm_mrmr, spatial_features_norm_mrmr], axis=1)

spatial_features_norm_mrmr.shape, X_norm_mrmr_with_spatial_features_norm_mrmr.shape

((420, 36), (420, 69))

## rLDA_model_params_grid_search

In [19]:
#oversample = SMOTE()
#X_smote, y_smote = oversample.fit_resample(X_with_spatial_features, y)
#Xs_train, Xs_test, ys_train, ys_test = train_test_split(X_smote, y_smote, test_size=0.30, random_state=42, shuffle=True, stratify=y_smote)
#X_with_spatial_features.shape, X_smote.shape, Xs_train.shape, Xs_test.shape

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_with_spatial_features, label, test_size=0.30, random_state=42, shuffle=True, stratify=label)
oversample = SMOTE()
Xs_train, ys_train = oversample.fit_resample(X_train, y_train)
#Xs_train, ys_train = X_train, y_train
Xs_test, ys_test = X_test, y_test
X.shape, X_train.shape, Xs_train.shape, Xs_test.shape

((420, 462), (294, 2964), (294, 2964), (126, 2964))

In [56]:
LDA_final=LinearDiscriminantAnalysis(shrinkage='auto', solver='eigen')
LDA_final.fit_transform(Xs_train, ys_train)
Xs_test_predictions=LDA_final.predict(Xs_test)

#ConfusionMatrixDisplay.from_predictions(y_test, X_test_predictions)
ConfusionMatrixDisplay.from_predictions(ys_test, Xs_test_predictions)

if type(y_test)==pd.DataFrame:
    tn, fp, fn, tp = confusion_matrix(list(y_test.values), list(Xs_test_predictions), labels=[0, 1]).ravel()
else:
    tn, fp, fn, tp = confusion_matrix(list(ys_test), list(Xs_test_predictions), labels=[0, 1]).ravel()

print('True Positive :', tp)
print('True Negative :', tn)
print('False Positive :', fp)
print('False Negative :', fn)
if type(y_test)==pd.DataFrame:
    print("Precision score",np.round(precision_score(y_test.values, Xs_test_predictions),3))
else:
    print("Precision score",np.round(precision_score(ys_test, Xs_test_predictions),3))
plt.show(block=True)

f:\Software\Anaconda\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


True Positive : 47
True Negative : 52
False Positive : 11
False Negative : 16
Precision score 0.81


In [21]:
import numpy as np
best_score = 0.0
best_params = None

def rLDA_model_params_grid_search(used_feature, label, shrinkage_range, min_gap=1e-5, log_path=None, random_state=None):
    #global best_score, best_params
    LDA = LinearDiscriminantAnalysis()

    cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=2, random_state=random_state)

    grid = dict()
    grid['solver'] = ['eigen']
    shrinkages = np.linspace(shrinkage_range[0], shrinkage_range[1], shrinkage_range[2]).tolist()
    #shrinkages = np.linspace(0.0, 1.0, 21).tolist()
    #shrinkages = np.linspace(0.1, 0.9, 9).tolist()
    #shrinkages = np.linspace(0.53, 0.55, 50).tolist()
    #shrinkages = np.linspace(0.01, 0.1, 10).tolist()
    #shrinkages = np.linspace(0.001, 0.01, 10).tolist()
    #shrinkages = np.linspace(0.000001, 0.00001, 10).tolist()

    grid['shrinkage'] = ['auto'] + shrinkages
    #scoring='precision'
    search = GridSearchCV(LDA, grid, scoring='accuracy', cv=cv, n_jobs=-1)

    # only_spatial_features  only_spatial_features_norm
    # X_with_spatial_features X_with_spatial_features_norm
    # X_norm_with_spatial_features_norm X_norm
    # only_spatial_features_norm_ave
    # X X_pca_norm
    results = search.fit(used_feature, label)
    log_xalg('result: %.4f' % results.best_score_, 'configuration:', results.best_params_, log_path=log_path)
    #if best_score == 0 and best_params is None:
    #    best_score = results.best_score_ 
    #    best_params = results.best_params_ 
    #elif results.best_score_ > best_score:
    #    best_score = results.best_score_ 
    #    best_params = results.best_params_ 
    #else:
    #    return best_score, best_params

    if results.best_params_['shrinkage'] == 'auto':
        log_xalg('within_class_shrinkages:', search.best_estimator_.within_class_shrinkages, 'tot_class_shrinkage:', search.best_estimator_.tot_class_shrinkage, log_path=log_path)
        good_score1 = results.best_score_ 
        good_params1 = results.best_params_ 
    else:
        gap = (shrinkage_range[1]-shrinkage_range[0]) / (shrinkage_range[2]-1)
        #log_xalg('gap:', gap, 'shrinkage_range:', shrinkage_range)
        if gap < min_gap:
            return results.best_score_ , results.best_params_
        shrinkage_range=(results.best_params_['shrinkage']-gap, results.best_params_['shrinkage']+gap, shrinkage_range[2])
        good_score1, good_params1 = rLDA_model_params_grid_search(used_feature, label, shrinkage_range, log_path=log_path)
        if good_score1 < results.best_score_ :
            good_score1 = results.best_score_ 
            good_params1 = results.best_params_ 
    return good_score1, good_params1

In [110]:
from itertools import product
from itertools import combinations
log_path=r'D:\BoyangDeng\test\exhuasted_search.txt'
if os.path.exists(log_path):
    os.remove(log_path)
features=spatial_features_norm

haralick_length_per_feature = spatial_features_num_dict['haralick_features']/len(band_list)
lbp_length_per_feature = spatial_features_num_dict['lbp_features']/len(band_list)
hu_moments_length_per_feature = spatial_features_num_dict['hu_moments_features']/len(band_list)
features_lengths = [haralick_length_per_feature, lbp_length_per_feature, hu_moments_length_per_feature]

start_idx_haralick = 0
start_idx_lbp = spatial_features_num_dict['haralick_features']
start_idx_hu_moments = (spatial_features_num_dict['haralick_features']+spatial_features_num_dict['lbp_features'])
features_start_idxes = [start_idx_haralick, start_idx_lbp, start_idx_hu_moments]

band_num = len(band_list)
band_idx_list = np.arange(0, band_num).tolist()
combine_num = 0
#band_combines = combinations(band_idx_list, r=6)
band_combines = combine_list_sort
#band_combines=[(0,1,3,4)]

selected_features_list = []
for band_combine in band_combines:
    if len(band_combine) == 0:
        continue
    combine_num += 1
    #log_xalg('band_combine:', band_combine, log_path=log_path)
    band_combine = np.array(band_combine)

    feature_idxes = []
    for features_length, features_start_idx in zip(features_lengths, features_start_idxes):
        for start_idx in band_combine*features_length:
            start_idx = int(start_idx)
            features_length = int(features_length)
            feature_idxes += list(range(features_start_idx+start_idx, features_start_idx+start_idx+features_length))
    selected_features = features[:, feature_idxes]
    selected_features_list.append(selected_features)
    #log_xalg('selected_features.shape', selected_features.shape, log_path=log_path)

    #seed_int = 0
    #np.random.seed(seed_int)
    ##np.random.seed(int(time.time()))
    #random_state = np.random.randint(0, 1e4)
    #shrinkage_range=(0.0, 1.0, 21)
    #good_score, good_params = rLDA_model_params_grid_search(selected_features, y, shrinkage_range, min_gap=1e-5, log_path=log_path, random_state=seed)
    #log_xalg('random_state: ', random_state,'good_score: %.3f' % good_score, 'good_params:', good_params, log_path=log_path)

    #np.random.seed(seed_int)
    #seed=True
    #model_params =best_params
    #xxx=repeated_tests(LinearDiscriminantAnalysis, only_spatial_features_norm, y, split_ratio=0.3, test_num=100, model_params=model_params, seed=seed, log_path=log_path)

log_xalg('len(selected_features_list)', len(selected_features_list), log_path=log_path)

len(selected_features_list) 63


In [316]:
def band_exhausted_search(band_list, features, y, spatial_features_num_dict, min_gap, log_path, seed):
    from itertools import product
    from itertools import combinations
    global best_score, best_params

    features=only_spatial_features_norm

    haralick_length_per_feature = spatial_features_num_dict['haralick_features']/len(band_list)
    lbp_length_per_feature = spatial_features_num_dict['lbp_features']/len(band_list)
    hu_moments_length_per_feature = spatial_features_num_dict['hu_moments_features']/len(band_list)
    features_lengths = [haralick_length_per_feature, lbp_length_per_feature, hu_moments_length_per_feature]

    start_idx_haralick = 0
    start_idx_lbp = spatial_features_num_dict['haralick_features']
    start_idx_hu_moments = (spatial_features_num_dict['haralick_features']+spatial_features_num_dict['lbp_features'])
    features_start_idxes = [start_idx_haralick, start_idx_lbp, start_idx_hu_moments]

    band_num = len(band_list)
    band_idx_list = np.arange(0, band_num).tolist()
    combine_num = 0

    good_score_list = []
    good_params_list = []
    combine_list = []
    if os.path.exists(log_path):
        os.remove(log_path)
    for i_num in range(band_num+1):
        band_combines = combinations(band_idx_list, r=i_num)
        for band_combine in band_combines:
            if len(band_combine) == 0:
                continue
            combine_num += 1
            log_xalg('band_combine:', band_combine, log_path=log_path)
            band_combine = np.array(band_combine)

            feature_idxes = []
            for features_length, features_start_idx in zip(features_lengths, features_start_idxes):
                for start_idx in band_combine*features_length:
                    start_idx = int(start_idx)
                    features_length = int(features_length)
                    feature_idxes += list(range(features_start_idx+start_idx, features_start_idx+start_idx+features_length))
            selected_features = features[:, feature_idxes]
            log_xalg('selected_features.shape', selected_features.shape, log_path=log_path)
            seed_int = 0
            np.random.seed(seed_int)
            #np.random.seed(int(time.time()))
            random_state = np.random.randint(0, 1e4)
            shrinkage_range=(0.0, 1.0, 21)
            good_score, good_params = rLDA_model_params_grid_search(selected_features, y, shrinkage_range, min_gap=1e-5, log_path=log_path, random_state=seed)
            log_xalg('random_state: ', random_state,'good_score: %.3f' % good_score, 'good_params:', good_params, log_path=log_path)

            good_score_list.append(good_score)
            good_params_list.append(good_params)
            combine_list.append(band_combine)
            #np.random.seed(seed)
            #seed=True
            #model_params=good_params
            #xxx=repeated_tests(LinearDiscriminantAnalysis, only_spatial_features_norm, y, split_ratio=0.3, test_num=100, model_params=model_params, seed=seed, log_path=log_path)
            #log_xalg('reset best_score, best_params', log_path=log_path)
            best_score = 0.0
            best_params = None
    log_xalg('combine_num:', combine_num, log_path=log_path)
    return good_score_list, good_params_list, combine_list

In [ ]:
log_path=r'F:\test\exhuasted_search.txt'
good_score_list, good_params_list, combine_list=band_exhausted_search(band_list, spatial_features_norm, y, spatial_features_num_dict, min_gap=1e-5, log_path=log_path, seed=0)

In [ ]:
good_score_list=np.array(good_score_list)
good_params_list=np.array(good_params_list)
combine_list=np.array(combine_list)
idxes=np.argsort(good_score_list)[::-1]
good_score_list_sort = good_score_list[idxes]
good_params_list_sort = good_params_list[idxes]
combine_list_sort = combine_list[idxes]

In [ ]:
repeated_tests(LinearDiscriminantAnalysis, X_with_spatial_features, label, split_ratio=0.3, test_num=100, model_params=None)

In [ ]:
repeated_tests_with_SMOTE(LinearDiscriminantAnalysis, X_with_spatial_features, label, split_ratio=0.3, test_num=100, model_params=None)

In [ ]:
model_params ={'shrinkage': 'auto', 'solver':'eigen'}
repeated_tests(LinearDiscriminantAnalysis, only_spatial_features_norm, label, split_ratio=0.3, test_num=100, model_params=model_params)

In [ ]:
np.random.seed(0)
seed=True
model_params ={'shrinkage': 6e-6, 'solver':'eigen'}
xxx=repeated_tests(LinearDiscriminantAnalysis, X_norm, label, split_ratio=0.3, test_num=100, model_params=model_params, seed=seed)

In [ ]:
np.random.seed(0)
seed=True
# precision
#model_params ={'shrinkage': 0.35, 'solver':'eigen'}
# accuracy
model_params ={'shrinkage': 0.3089, 'solver':'eigen'}
xxx=repeated_tests(LinearDiscriminantAnalysis, only_spatial_features_norm, label, split_ratio=0.3, test_num=10, model_params=model_params, seed=seed)

In [ ]:
np.random.seed(0)
seed=True
model_params ={'shrinkage': 0.1722, 'solver':'eigen'}
repeated_tests(LinearDiscriminantAnalysis, only_spatial_features_norm_ave, label, split_ratio=0.3, test_num=100, model_params=model_params, seed=seed)

In [ ]:
"""
21 bands
"""
np.random.seed(0)
seed=True
model_params ={'shrinkage': 0.62, 'solver':'eigen'}
xxx=repeated_tests(LinearDiscriminantAnalysis, only_spatial_features_norm, label, split_ratio=0.3, test_num=100, model_params=model_params, seed=seed)

In [29]:
"""
6 bands / 3 bands / 5 bands
"""
np.random.seed(0)
seed=True
model_params ={'shrinkage': 0.363, 'solver':'eigen'}
#model_params ={'shrinkage': 0.2716, 'solver':'eigen'}
#model_params ={'shrinkage': 0.5349, 'solver':'eigen'}
xxx=repeated_tests(LinearDiscriminantAnalysis, only_spatial_features_norm, label, split_ratio=0.3, test_num=100, model_params=model_params, seed=seed)

100%|██████████| 100/100 [00:02<00:00, 49.81it/s]

(420, 276) (294, 276) (126, 276) 

accuracy:
                0
count  100.000000
mean     0.874127
std      0.026262
min      0.801587
25%      0.857143
50%      0.880952
75%      0.888889
max      0.936508


In [61]:
tmp = X_norm_with_spatial_features_norm.copy()
X_norm_with_spatial_features_norm_shuffle = tmp.transpose(1,0)
np.random.shuffle(X_norm_with_spatial_features_norm_shuffle)
X_norm_with_spatial_features_norm_shuffle = X_norm_with_spatial_features_norm_shuffle.transpose(1,0)
X_norm_with_spatial_features_norm_shuffle.shape
idxes = np.where(X_norm_with_spatial_features_norm_shuffle != X_norm_with_spatial_features_norm)
len(idxes[0])

252000

In [74]:
"""
Precision: 0.939
'shrinkage': 8.5789e-05
"""
#X_norm_with_spatial_features_norm_scale = np.concatenate([X_norm, 0.1*all_spatial_features_norm[:, start: end]], axis=1)
#X_pca_norm_with_spatial_features_norm = np.concatenate([X_pca_norm, only_spatial_features_norm], axis=1)
X_pca_norm_with_spatial_features_pca_norm = np.concatenate([X_pca_norm, results['PC'].values], axis=1)
X_pca_norm_with_spatial_features_pca_norm.shape

(420, 52)

In [ ]:
#from sklearn.decomposition import PCA
#pca = PCA(n_components=20)
#X_pca_spatial = pca.fit_transform(only_spatial_features_norm)
#X_pca_spatial.shape
#pca.explained_variance_

from pca import pca
model = pca(n_components=0.95)
results = model.fit_transform(only_spatial_features_norm)
results.keys(), results['PC'].values.shape

In [ ]:
np.linspace(0.35, 0.37, 20).tolist()

In [ ]:
model_params ={'shrinkage':'auto', 'solver':'eigen'}
repeated_tests_with_SMOTE(LinearDiscriminantAnalysis, X_with_spatial_features, label, split_ratio=0.3, test_num=100, model_params=model_params)

In [ ]:
np.random.seed(0)
seed=True
model_params =[{'shrinkage': 6e-6, 'solver':'eigen'}, {'shrinkage': 0.3089, 'solver':'eigen'}]
model_class = [LinearDiscriminantAnalysis, LinearDiscriminantAnalysis]
data = [X_norm, only_spatial_features_norm]
precisions, recalls, accuracies = repeated_tests(model_class, data, label, split_ratio=0.3, test_num=200, model_params=model_params, seed=seed)

In [56]:
import scipy.stats as stats
stats.ttest_rel(accuracies[0], accuracies[1])

Ttest_relResult(statistic=55.323994137780055, pvalue=8.702115684323326e-123)

## bands exhuasted search

In [ ]:
# code in rLDA + image features part

## sprectral/spatial features -> ensemble

In [19]:
#random_state=42
random_state = np.random.randint(0, 1e4)
X_train, X_test, y_train, y_test = train_test_split(X_norm, label, test_size=0.30, random_state=random_state, shuffle=True, stratify=label)
X_train1, X_test1, y_train1, y_test1 = train_test_split(spatial_features_norm, label, test_size=0.30, random_state=random_state, shuffle=True, stratify=label)
X_train.shape, X_train1.shape

((294, 462), (294, 2502))

In [20]:
#random_state = 7313
#random_state = np.random.randint(0, 1e4)
random_state = 0
print('random_state:', random_state)
X_train, X_test, y_train, y_test = train_test_split(X_norm, label, test_size=0.30, random_state=random_state, shuffle=True, stratify=label)
X_train1, X_test1, y_train1, y_test1 = train_test_split(spatial_features_norm, label, test_size=0.30, random_state=random_state, shuffle=True, stratify=label)
y_train = y_train.squeeze()
y_train1 = y_train1.squeeze()

model_class=LinearDiscriminantAnalysis
#model_params ={'shrinkage': 6e-6, 'solver':'eigen'}
model_params ={'shrinkage': 5e-6, 'solver':'eigen'}
model_spectral = model_class(**model_params)

#model_params1 ={'shrinkage': 0.35, 'solver':'eigen'}
model_params1 ={'shrinkage': 1e-06, 'solver':'eigen'}
model_spatial = model_class(**model_params1)

model_spectral.fit(X_train, y_train)
model_spatial.fit(X_train1, y_train1)

predictions = model_spectral.predict(X_test)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
print('model_spectral precision:', precision)
print('model_spectral recall:', recall)

predictions1 = model_spatial.predict(X_test1)
precision = precision_score(y_test1, predictions1)
recall = recall_score(y_test1, predictions1)
print('model_spatial precision:', precision)
print('model_spatial recall:', recall)

diffs = np.where(predictions!=y_test.values.squeeze())
print('accuracy:', 1-len(diffs[0])/len(y_test), diffs)

diffs=np.where(predictions1!=y_test1.values.squeeze())
print('accuracy:', 1-len(diffs[0])/len(y_test), diffs)

random_state: 0
model_spectral precision: 0.967741935483871
model_spectral recall: 0.9523809523809523
model_spatial precision: 0.8888888888888888
model_spatial recall: 0.7619047619047619
accuracy: 0.9603174603174603 (array([ 32,  41,  93, 104, 118], dtype=int64),)
accuracy: 0.8333333333333334 (array([  8,  10,  16,  43,  44,  56,  59,  63,  66,  70,  71,  77,  85,
        87,  89,  93,  96, 100, 109, 110, 118], dtype=int64),)


In [ ]:
score_sprectral = model_spectral.decision_function(X_train)
df_describe = pd.DataFrame(score_sprectral)
print(df_describe.describe())
statistics = df_describe.describe()
mean_spectral = statistics.loc['mean'].values
std_spectral = statistics.loc['std'].values

In [ ]:
score_spatial = model_spatial.decision_function(X_train1)
df_describe = pd.DataFrame(score_spatial)
print(df_describe.describe())
statistics = df_describe.describe()
mean_spatial = statistics.loc['mean'].values
std_spatial = statistics.loc['std'].values

In [326]:
score_spectral = model_spectral.decision_function(X_test)
score_spatial = model_spatial.decision_function(X_test1)

In [ ]:
predictions_final = []
i_scores = np.array([  4,  40,  55,  58, 111, 121])
i_scores = np.array([  4,   7,  20,  24,  29,  36,  40,  41,  79,  84,  86,  88,  92, 99, 114])
for i_score in i_scores:
    score_spectral_norm = (score_spectral[i_score]-mean_spectral)/std_spectral
    score_spatial_norm = (score_spatial[i_score]-mean_spatial)/std_spatial
    bias = 0.4
    if score_spatial_norm > 0:
        if abs(score_spectral_norm) > abs(score_spatial_norm) - bias:
            final_score = score_spectral_norm
        else:
            final_score = score_spatial_norm
    else:
        final_score = score_spectral_norm

    if final_score > 0:
        final_cls = 1
    else:
        final_cls = 0
    predictions_final.append(final_cls)
    print('i_score:', i_score)
    print('score_spectral_norm:', score_spectral_norm)
    print('score_spatial_norm:', score_spatial_norm)
    print((y_test.values[i_score]==final_cls)[0], ', y:', y_test.values[i_score], ', final_cls:', final_cls)
    print('')

In [296]:
"""
use abs(score_spectral_norm) - abs(score_spatial_norm) as x
"""
def diff_trust(x):
    trust_level=[0,0]
    if x<-0.4:
        trust_level[0]=abs(x+0.4)
        trust_level[1]=0
    if x>=-0.4:
        trust_level[0]=0
        trust_level[1]=abs(x+0.4)
    return trust_level

In [297]:
"""
use abs(score_spectral_norm) and abs(score_spatial_norm) as x1, x2
low trust / high trust
"""
def spectral_trust(x):
    x = abs(x)
    trust_level=[0,0]
    if 0.0<=x<0.7:
        trust_level[0]=(0.7-x)/0.7 + 0.5
        trust_level[1]=0.0
    if 0.7<=x<=1.3:
        trust_level[0]=0.5*(-5/3*x+13/6)
        trust_level[1]=0.5*(x-0.7)/0.6
    elif 1.3<x<=1e6:
        trust_level[0]=0.0
        trust_level[1]=(x-1.3) + 0.5
    return trust_level

def spatial_trust(x):
    x = abs(x)
    trust_level=[0,0]
    if 0.0<=x<0.7:
        trust_level[0]=(0.7-x)/0.7 + 0.5
        trust_level[1]=0.0
    if 0.7<=x<=1.3:
        trust_level[0]=0.5*(-5/3*x+13/6)
        trust_level[1]=0.5*(x-0.7)/0.6
    elif 1.3<x<=1e6:
        trust_level[0]=0.0
        trust_level[1]=(x-1.3) + 0.5
    return trust_level

In [298]:
"""
every place can sum to 1.0

-3.0*std->3.0*std to 0.0->1.0
"""
def spectral_level(x):
    level=[0,0]
    level[0]=max((1.0-x), 0)
    level[1]=max(x, 0)
    return level

def spatial_level(x):
    level=[0,0]
    level[0]=max((1.0-x), 0)
    level[1]=max(x, 0)
    return level

In [299]:
"""
every place can sum to 1.0

0: good
1: bad
"""
def quality_level(x):
    level=[0,0]
    level[0]=max((1.0-x), 0)
    level[1]=max(x, 0)
    return level

In [301]:
def rules(spectral, spatial):
    """
    0: trust spectral
    1: trust spectral
    2: trust spatial
    3: trust spectral
    """
    rule_values = [0, 0, 0, 0]
    rule_values[0]=min(spectral[0], spatial[0])
    rule_values[1]=min(spectral[1], spatial[0])
    rule_values[2]=min(spectral[0], spatial[1])
    rule_values[3]=min(spectral[1], spatial[1])
    return rule_values

def infer(rule_values, spectral, spatial, bias=0):
    quality_level = [0, 0]
    trust_spectral = max(rule_values[0], rule_values[1], rule_values[3])+bias
    trust_spatial = rule_values[2]
    quality_level[0]=spectral[0]*trust_spectral + spatial[0]*trust_spatial
    quality_level[1]=spectral[1]*trust_spectral + spatial[1]*trust_spatial
    return quality_level

def area_gravity(quality_level):
    confidence=[0,0]
    confidence[0]=1.0 - quality_level[0]
    confidence[1]=quality_level[1]
    sum_1=confidence[0]*quality_level[0]+confidence[1]*quality_level[1]
    sum_2=quality_level[0]+quality_level[1]
    final_confidence=sum_1/sum_2
    return final_confidence

def maximum_membership(confidence):
    good=1.0-confidence
    bad=confidence
    if(bad>good):
        #'bad'
        quality_level=1
    else:
        # 'good'
        quality_level=-1
    return quality_level

In [302]:
def fuzzy_logic_infer(spectral_value, spatial_value, bias=0.4):
    spectral=spectral_level(spectral_value)
    spatial=spatial_level(spatial_value)
    rules_value=rules(spectral, spatial)
    quality_level=infer(rules_value, spectral, spatial, bias)
    result_1=area_gravity(quality_level)
    result_2=maximum_membership(result_1)
    #print("class: {}, predict confidence: {}".format(result_2,int(result_1+0.5)))
    return result_2
spectral_value=0.4
spatial_value=0.6
fuzzy_logic_infer(spectral_value, spatial_value)

-1

In [303]:
x=np.linspace(-0.5,-0.3,100)
y=[]
y1=[]
for i in x:
    j,k=diff_trust(i)
    y.append(j)
    y1.append(k)
plt.plot(x, y1)

In [329]:
predictions_final = []
for i_score, _ in enumerate(score_spectral):
    score_spectral_norm = (score_spectral[i_score]-mean_spectral)/std_spectral
    score_spatial_norm = (score_spatial[i_score]-mean_spatial)/std_spatial

    bias = 0.4
    if score_spatial_norm > 0:
        if abs(score_spectral_norm) > abs(score_spatial_norm) - bias:
            final_score = score_spectral_norm
        else:
            final_score = score_spatial_norm
    else:
        final_score = score_spectral_norm

    if final_score > 0:
        final_cls = 1
    else:
        final_cls = 0
    predictions_final.append(final_cls)

    #print('i_score:', i_score)
    #print('score_spectral_norm:', score_spectral_norm)
    #print('score_spatial_norm:', score_spatial_norm)
    #print((y_test.values[i_score]==final_cls)[0], ', y:', y_test.values[i_score], ', final_cls:', final_cls)
    #print('')

#np.where(predictions!=y_test.values.squeeze())
diffs=np.where(predictions_final!=y_test.values.squeeze())
1-len(diffs[0])/len(y_test), diffs

(0.9603174603174603, (array([ 41,  66,  93, 104, 118], dtype=int64),))

### repeated_test_ensamble

In [21]:
#random_state = 100
random_state = 0
X_train, X_test, y_train, y_test = train_test_split(X_norm, label, test_size=0.30, random_state=random_state, shuffle=True, stratify=label)

X_train1, X_test1, y_train1, y_test1 = train_test_split(X_norm, label, test_size=0.30, random_state=random_state, shuffle=True, stratify=label)
np.where(X_train != X_train1)

(array([], dtype=int64), array([], dtype=int64))

In [22]:
"""ensemble_method:
    statistics_driven
    fuzzy_logic
"""
# shrinkage_spectral=6e-6, shrinkage_spatial=0.363
# shrinkage_spectral=5e-6, shrinkage_spatial=1e-6
def repeated_test_ensamble(model_spectral, model_spatial, feature1, feature2, label, test_num=100, seed=None, model_params_spectral=None, model_params_spatial=None, ensemble_method='statistics_driven'):
    from tqdm import tqdm
    accuracy_spectrals = []
    accuracy_spatials = []
    accuracy_ensemble = []

    precision_spectrals = []
    precision_spatials = []
    precision_ensemble = []

    recall_spectrals = []
    recall_spatials = []
    recall_ensemble = []

    all_score_spectrals_train = []
    all_score_spatials_train = []

    all_score_spectral_norms_train = []
    all_score_spatial_norms_train = []

    all_pred_spectral_norms_train = []
    all_pred_spatial_norms_train = []

    all_score_spectrals_test = []
    all_score_spatials_test = []

    all_score_spectral_norms_test = []
    all_score_spatial_norms_test = []

    all_pred_spectral_norms_test = []
    all_pred_spatial_norms_test = []

    all_y_test = []
    all_y_train = []

    all_bias = []

    if 'PLS' in model_spectral.__name__ or 'PLS' in model_spatial.__name__:
        boundary_center = 0.5

    #print('feature1.shape', feature1.shape)
    #print('feature2.shape', feature2.shape)
    for i_test in tqdm(range(test_num)):
        if seed is None:
            np.random.seed(int(time.time()))
        random_state = np.random.randint(0, 1e4)
        X_train, X_test, y_train, y_test = train_test_split(feature1, label, test_size=0.30, random_state=random_state, shuffle=True, stratify=label)
        y_train = y_train.squeeze()
        y_test = y_test.squeeze()

        if type(feature2) is list:
            X_train1_list=[]
            X_test1_list=[]
            y_train1_list=[]
            y_test1_list=[]
            for feature2_ in feature2:
                X_train1_, X_test1_, y_train1_, y_test1_ = train_test_split(feature2_, label, test_size=0.30, random_state=random_state, shuffle=True, stratify=label)
                y_train1_ = y_train1_.squeeze()
                y_test1_ = y_test1_.squeeze()
                X_train1_list.append(X_train1_)
                X_test1_list.append(X_test1_)
                y_train1_list.append(y_train1_)
                y_test1_list.append(y_test1_)
        else:
            X_train1, X_test1, y_train1, y_test1 = train_test_split(feature2, label, test_size=0.30, random_state=random_state, shuffle=True, stratify=label)
            y_train1 = y_train1.squeeze()
            y_test1 = y_test1.squeeze()

        if model_params_spectral is not None:
            model_ = model_spectral(**model_params_spectral)
        else:
            model_ = model_spectral()
        if 'PLS' in model_spectral.__name__:
            y_train = pd.get_dummies(y_train)

        model_.fit(X_train, y_train)
        predictions = model_.predict(X_test)
        
        if 'PLS' in model_spectral.__name__:
            predictions = np.array([np.argmax(i) for i in predictions])

        precision = precision_score(y_test, predictions)
        recall = recall_score(y_test, predictions)
        diffs = np.where(predictions!=y_test.values.squeeze())
        accuracy_spectral = 1-len(diffs[0])/len(y_test)

        precision_spectrals.append(precision)
        recall_spectrals.append(recall)
        accuracy_spectrals.append(accuracy_spectral)
        if 'PLS' in model_spectral.__name__:
            score_spectral_train = model_.predict(X_train)
            score_spectral_train = np.max(score_spectral_train, 1) - boundary_center
            score_spectral_train[score_spectral_train<=0] = 0
        else:
            score_spectral_train = model_.decision_function(X_train)
        df_describe = pd.DataFrame(score_spectral_train)
        statistics = df_describe.describe()
        mean_spectral_train = statistics.loc['mean'].values
        std_spectral_train = statistics.loc['std'].values

        #model_params1 ={'shrinkage': 0.35, 'solver':'eigen'}
        #model_params1 ={'shrinkage': 0.62, 'solver':'eigen'}
        #model_params1 ={'shrinkage': 0.363, 'solver':'eigen'}
        #model_params1 ={'shrinkage': 0.5349, 'solver':'eigen'}
        if type(feature2) is list:
            score_spatial_train = None
            precision_spatial_bands = []
            recall_spatial_bands = []
            accuracy_spatial_bands = []
            model_spatial_all = []
            assert type(model_params_spectral) is list
            for i_model, model_param in enumerate(model_params_spectral):
                model_params1 = model_param
                if model_params1 is not None:
                    model_1 = model_spatial(**model_params1)
                else:
                    model_1 = model_spatial()
                if 'PLS' in model_spatial.__name__:
                    y_train1_list[i_model] = pd.get_dummies(y_train1_list[i_model])

                model_1.fit(X_train1_list[i_model], y_train1_list[i_model])
                predictions1 = model_1.predict(X_test1_list[i_model])

                if 'PLS' in model_spatial.__name__:
                    predictions1 = np.array([np.argmax(i) for i in predictions1])

                precision = precision_score(y_test1_list[i_model], predictions1)
                recall = recall_score(y_test1_list[i_model], predictions1)
                diffs=np.where(predictions1!=y_test1_list[i_model].values.squeeze())
                accuracy_spatial = 1-len(diffs[0])/len(y_test1_list[i_model])
                precision_spatial_bands.append(precision)
                recall_spatial_bands.append(recall)
                accuracy_spatial_bands.append(accuracy_spatial)

                if score_spatial_train is None:
                    score_spatial_train = model_1.decision_function(X_train1_list[i_model])
                else:
                    score_spatial_train += model_1.decision_function(X_train1_list[i_model])
                model_spatial_all.append(model_1)

            precision_spatials.append(precision_spatial_bands)
            recall_spatials.append(recall_spatial_bands)
            accuracy_spatials.append(accuracy_spatial_bands)

            df_describe = pd.DataFrame(score_spatial_train)
            statistics = df_describe.describe()
            mean_spatial_train = statistics.loc['mean'].values
            std_spatial_train = statistics.loc['std'].values
        else:
            model_params1 = model_params_spatial
            if model_params1 is not None:
                model_1 = model_spatial(**model_params1)
            else:
                model_1 = model_spatial()
            if 'PLS' in model_spatial.__name__:
                y_train1 = pd.get_dummies(y_train1)

            model_1.fit(X_train1, y_train1)
            predictions1 = model_1.predict(X_test1)

            if 'PLS' in model_spatial.__name__:
                predictions1 = np.array([np.argmax(i) for i in predictions1])

            precision = precision_score(y_test1, predictions1)
            recall = recall_score(y_test1, predictions1)
            diffs=np.where(predictions1!=y_test1.values.squeeze())
            accuracy_spatial = 1-len(diffs[0])/len(y_test1)
            if 'PLS' in model_spatial.__name__:
                score_spatial_train = model_1.predict(X_train1)
                score_spatial_train = np.max(score_spatial_train, 1) - boundary_center
                score_spatial_train[score_spatial_train<=0] = 0
            else:
                score_spatial_train = model_1.decision_function(X_train1)

            df_describe = pd.DataFrame(score_spatial_train)
            statistics = df_describe.describe()
            mean_spatial_train = statistics.loc['mean'].values
            std_spatial_train = statistics.loc['std'].values

            precision_spatials.append(precision)
            recall_spatials.append(recall)
            accuracy_spatials.append(accuracy_spatial)

        score_spectrals = []
        score_spatials = []
        score_spectral_norms = []
        score_spatial_norms = []
        pred_spectral_norms = []
        pred_spatial_norms = []
        for i_score, _ in enumerate(score_spectral_train):
            score_spectral_norm = (score_spectral_train[i_score]-mean_spectral_train)/std_spectral_train
            score_spatial_norm = (score_spatial_train[i_score]-mean_spatial_train)/std_spatial_train

            score_spectrals.append(score_spectral_train[i_score])
            score_spatials.append(score_spatial_train[i_score])

            score_spectral_norms.append(score_spectral_norm)
            score_spatial_norms.append(score_spatial_norm)

            pred_spectral_norm = 1 if score_spectral_norm > 0 else 0
            pred_spatial_norm = 1 if score_spatial_norm > 0 else 0

            pred_spectral_norms.append(pred_spectral_norm)
            pred_spatial_norms.append(pred_spatial_norm)

        score_spectrals = np.array(score_spectrals)
        score_spatials = np.array(score_spatials)
        all_score_spectrals_train.append(score_spectrals)
        all_score_spatials_train.append(score_spatials)

        score_spectral_norms = np.array(score_spectral_norms)
        score_spatial_norms = np.array(score_spatial_norms)
        all_score_spectral_norms_train.append(score_spectral_norms)
        all_score_spatial_norms_train.append(score_spatial_norms)

        pred_spectral_norms = np.array(pred_spectral_norms)
        pred_spatial_norms = np.array(pred_spatial_norms)
        all_pred_spectral_norms_train.append(pred_spectral_norms)
        all_pred_spatial_norms_train.append(pred_spatial_norms)

        all_y_train.append(y_train)
        if 'PLS' in model_spectral.__name__:
            score_spectral_test = model_.predict(X_test)
            score_spectral_test = np.max(score_spectral_test, 1) - boundary_center
            score_spectral_test[score_spectral_test<=0] = 0
        else:
            score_spectral_test = model_.decision_function(X_test)
        
        if type(feature2) is list:
            score_spatial_test = None
            for i_model, _ in enumerate(X_test1_list):
                if score_spatial_test is None:
                    score_spatial_test = model_spatial_all[i_model].decision_function(X_test1_list[i_model])
                else:
                    score_spatial_test += model_spatial_all[i_model].decision_function(X_test1_list[i_model])
        else:
            if 'PLS' in model_spatial.__name__:
                score_spatial_test = model_1.predict(X_test1)
                score_spatial_test = np.max(score_spatial_test, 1) - boundary_center
                score_spatial_test[score_spatial_test<=0] = 0
            else:
                score_spatial_test = model_1.decision_function(X_test1)

        score_spectrals = []
        score_spatials = []
        score_spectral_norms = []
        score_spatial_norms = []
        pred_spectral_norms = []
        pred_spatial_norms = []

        predictions_final = []
        #if type(feature2) is list:
        #    bias = np.array(score_spatial_train).reshape(-1).std() / len(feature2) / np.array(score_spectral_train).reshape(-1).std()
        #else:
        #    bias = np.array(score_spatial_train).reshape(-1).std() / np.array(score_spectral_train).reshape(-1).std()
        bias = mean_spatial_train - mean_spectral_train
        all_bias.append(bias)
        for i_score, _ in enumerate(score_spectral_test):
            score_spectral_norm = (score_spectral_test[i_score]-mean_spectral_train)/std_spectral_train
            score_spatial_norm = (score_spatial_test[i_score]-mean_spatial_train)/std_spatial_train

            score_spectrals.append(score_spectral_test[i_score])
            score_spatials.append(score_spatial_test[i_score])

            score_spectral_norms.append(score_spectral_norm)
            score_spatial_norms.append(score_spatial_norm)

            pred_spectral_norm = 1 if score_spectral_norm > 0 else 0
            pred_spatial_norm = 1 if score_spatial_norm > 0 else 0

            pred_spectral_norms.append(pred_spectral_norm)
            pred_spatial_norms.append(pred_spatial_norm)

            if np.sign(score_spectral_norm) != np.sign(score_spatial_norm):
                if ensemble_method == 'statistics_driven':
                    # this bias more like SVM (hinge loss) than fuzzy logic
                    #bias = 0.22
                    if score_spatial_norm > 0:
                        if abs(score_spectral_norm) + bias > abs(score_spatial_norm):
                            final_score = score_spectral_norm
                        else:
                            final_score = score_spatial_norm
                    else:
                        final_score = score_spectral_norm
                #if ensemble_method == 'svc_model':
                    #res = svc_model.predict([[score_spectral_norm[0], score_spatial_norm[0]]])
                    #final_score = score_spectral_norm if res ==0 else score_spatial_norm
                elif ensemble_method == 'fuzzy_logic':
                    # bias=0.62
                    spectral_value = ((score_spectral_norm/3.0)+1.0)/2.0
                    spatial_value = ((score_spatial_norm/3.0)+1.0)/2.0
                    final_score = fuzzy_logic_infer(spectral_value, spatial_value, bias=3*bias)
                else:
                    assert False
            else:
                final_score = score_spectral_norm

            if final_score > 0:
                final_cls = 1
            else:
                final_cls = 0
            predictions_final.append(final_cls)

        score_spectrals = np.array(score_spectrals)
        score_spatials = np.array(score_spatials)
        all_score_spectrals_test.append(score_spectrals)
        all_score_spatials_test.append(score_spatials)

        score_spectral_norms = np.array(score_spectral_norms)
        score_spatial_norms = np.array(score_spatial_norms)
        all_score_spectral_norms_test.append(score_spectral_norms)
        all_score_spatial_norms_test.append(score_spatial_norms)

        pred_spectral_norms = np.array(pred_spectral_norms)
        pred_spatial_norms = np.array(pred_spatial_norms)
        all_pred_spectral_norms_test.append(pred_spectral_norms)
        all_pred_spatial_norms_test.append(pred_spatial_norms)

        all_y_test.append(y_test)

        precision = precision_score(y_test, predictions_final)
        recall = recall_score(y_test, predictions_final)

        diffs=np.where(predictions_final!=y_test.values.squeeze())
        final_accuracy = 1-len(diffs[0])/len(y_test)

        precision_ensemble.append(precision)
        recall_ensemble.append(recall)
        accuracy_ensemble.append(final_accuracy)
    return precision_spectrals, precision_spatials, precision_ensemble, recall_spectrals, recall_spatials, recall_ensemble, accuracy_spectrals, accuracy_spatials, accuracy_ensemble, all_y_train, all_score_spectral_norms_train, all_score_spatial_norms_train,  all_pred_spectral_norms_train, all_pred_spatial_norms_train, all_score_spectrals_train, all_score_spatials_train, all_y_test, all_score_spectral_norms_test, all_score_spatial_norms_test,  all_pred_spectral_norms_test, all_pred_spatial_norms_test, all_score_spectrals_test, all_score_spatials_test, all_bias

In [ ]:
final_accuracy_list = []
spectral_accuracy_list = []
spatial_accuracy_list = []

score_t = 0.875
idxes = np.where(good_score_list_sort >= score_t)[0]
print('candidate num:', len(idxes))
for i_select in range(len(selected_features_list)):
    if good_score_list_sort[i_select] < score_t:
        break
    feature1 = X_norm
    #feature2 = only_spatial_features_norm
    feature2 = selected_features_list[i_select]
    combine = combine_list_sort[i_select]
    shrinkage_spectral = 6e-6
    shrinkage_spatial = good_params_list_sort[i_select]['shrinkage']
    print('i_select:', i_select, ', combine:', combine, ', shrinkage_spatial:', shrinkage_spatial, ', score:', good_score_list_sort[i_select])

    np.random.seed(0)
    seed = True
    (precision_spectrals, precision_spatials, precision_ensemble, recall_spectrals, recall_spatials, recall_ensemble, accuracy_spectrals, accuracy_spatials, accuracy_ensemble, all_y_train, all_score_spectral_norms_train, all_score_spatial_norms_train,  all_pred_spectral_norms_train, all_pred_spatial_norms_train, all_score_spectrals_train, all_score_spatials_train, all_y_test, all_score_spectral_norms_test, all_score_spatial_norms_test,  all_pred_spectral_norms_test, all_pred_spatial_norms_test, all_score_spectrals_test, all_score_spatials_test, all_bias
    )=repeated_test_ensamble(feature1, feature2, label, test_num=100, seed=seed, shrinkage_spectral=shrinkage_spectral, shrinkage_spatial=shrinkage_spatial)
    accuracy_ensemble=np.array(accuracy_ensemble)
    accuracy_spectrals=np.array(accuracy_spectrals)
    accuracy_spatials=np.array(accuracy_spatials)
    mean_accuracy_final = accuracy_ensemble.mean()
    mean_accuracy_spectral = accuracy_spectrals.mean()
    mean_accuracy_spatial = accuracy_spatials.mean()
    print('mean_accuracy_final:', mean_accuracy_final)
    final_accuracy_list.append(mean_accuracy_final)
    spectral_accuracy_list.append(mean_accuracy_spectral)
    spatial_accuracy_list.append(mean_accuracy_spatial)
final_accuracy_list = np.array(final_accuracy_list)
idxes = np.argsort(final_accuracy_list)[::-1]
#idx = np.argmax(final_accuracy_list)
for idx in idxes:
    shrinkage = good_params_list_sort[idx]['shrinkage']
    if shrinkage != 'auto':
        shrinkage = round(shrinkage, 5)
    print("%2d"%idx, "%.4f"%round(final_accuracy_list[idx],4), "%.4f"%round(spectral_accuracy_list[idx], 4), "%.4f"%round(spatial_accuracy_list[idx], 4), combine_list_sort[idx], shrinkage)

In [ ]:
idxes = np.argsort(final_accuracy_list)[::-1]
print('candidate num:', len(good_score_list_sort))
for i_rank, idx in enumerate(idxes):
    shrinkage = good_params_list_sort[idx]['shrinkage']
    if shrinkage != 'auto':
        shrinkage = round(shrinkage, 5)
    print("%2d"%i_rank, "%2d"%idx, "%.4f"%round(final_accuracy_list[idx],4), "%.4f"%round(spectral_accuracy_list[idx], 4), "%.4f"%round(spatial_accuracy_list[idx], 4), combine_list_sort[idx], shrinkage)

In [25]:
from sklearn.cross_decomposition import PLSRegression


shrinkage_spectral = 5e-6
shrinkage_spatial = 'auto'
#i_select=8
#feature2 = selected_features_list[i_select]
#combine = combine_list_sort[i_select]
#shrinkage_spatial = good_params_list_sort[i_select]['shrinkage']

seed_int=0
#seed_int=1673588891  
#seed_int = int(time.time())
np.random.seed(seed_int)
seed = True

#print('seed_int:', seed_int, 'combine:', combine, ', shrinkage_spatial:', round(shrinkage_spatial,5), ', score:', round(good_score_list_sort[i_select], 4))

# statistics_driven
# fuzzy_logic

model_params_spectral={'shrinkage': shrinkage_spectral, 'solver': 'eigen'}
#model_params_spectral={'n_components': 28,}
#model_params_spectral=None

#model_params_spatial={'shrinkage': shrinkage_spatial, 'solver': 'eigen'}
model_params_spatial=None
#model_params_spatial={'n_components': 5,}

# LinearDiscriminantAnalysis

# 28 5 6 22 1
#model_params={'n_components': 1,}
#PLSRegression

spectral_model = LinearDiscriminantAnalysis
spatial_model = LinearDiscriminantAnalysis

#X_norm
#spatial_features_norm
#X_norm_with_spatial_features_norm
#X_norm_with_spatial_features_norm_MRMR_norm
#X_norm_with_spatial_features_norm_PCA_norm
#X_norm_mrmr
#spatial_features_mrmr
#spatial_features_norm_MRMR

feature1 = X_norm
feature2 = spatial_features_norm

(precision_spectrals, precision_spatials, precision_ensemble, recall_spectrals, recall_spatials, recall_ensemble, accuracy_spectrals, accuracy_spatials, accuracy_ensemble, all_y_train, all_score_spectral_norms_train, all_score_spatial_norms_train,  all_pred_spectral_norms_train, all_pred_spatial_norms_train, all_score_spectrals_train, all_score_spatials_train, all_y_test, all_score_spectral_norms_test, all_score_spatial_norms_test,  all_pred_spectral_norms_test, all_pred_spatial_norms_test, all_score_spectrals_test, all_score_spatials_test, all_bias
)=repeated_test_ensamble(spectral_model, spatial_model, feature1, feature2, label, test_num=100, seed=seed, model_params_spectral=model_params_spectral, model_params_spatial=model_params_spatial, ensemble_method='statistics_driven')

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:28<00:00,  3.45it/s]


In [147]:
i_select=20
feature1 = X_norm
#feature2 = only_spatial_features_norm
feature2 = selected_features_list[:i_select]
combine = combine_list_sort[:i_select]
shrinkage_spectral = 6e-6
#shrinkage_spatial = good_params_list_sort[:i_select]['shrinkage']
shrinkage_spatial = []
for good_params in good_params_list_sort[:i_select]:
    shrinkage_spatial += [good_params['shrinkage']]

seed_int=0
#seed_int=1673588746
#seed_int = int(time.time())
np.random.seed(seed_int)

seed = True
(precision_spectrals, precision_spatials, precision_ensemble, recall_spectrals, recall_spatials, recall_ensemble, accuracy_spectrals, accuracy_spatials, accuracy_ensemble, all_y_train, all_score_spectral_norms_train, all_score_spatial_norms_train,  all_pred_spectral_norms_train, all_pred_spatial_norms_train, all_score_spectrals_train, all_score_spatials_train, all_y_test, all_score_spectral_norms_test, all_score_spatial_norms_test,  all_pred_spectral_norms_test, all_pred_spatial_norms_test, all_score_spectrals_test, all_score_spatials_test, all_bias
)=repeated_test_ensamble(feature1, feature2, label, test_num=100, seed=seed, shrinkage_spectral=shrinkage_spectral, shrinkage_spatial=shrinkage_spatial)


100%|██████████| 100/100 [00:36<00:00,  2.75it/s]


In [26]:
show_statistics(precision_ensemble), show_statistics(recall_ensemble), show_statistics(accuracy_ensemble)

                0
count  100.000000
mean     0.964841
std      0.025180
min      0.887324
25%      0.953125
50%      0.967742
75%      0.983871
max      1.000000
                0
count  100.000000
mean     0.967143
std      0.023365
min      0.888889
25%      0.952381
50%      0.968254
75%      0.984127
max      1.000000
                0
count  100.000000
mean     0.965556
std      0.016120
min      0.928571
25%      0.952381
50%      0.968254
75%      0.976190
max      1.000000


(None, None, None)

In [27]:
show_statistics(precision_spectrals), show_statistics(precision_spatials), show_statistics(precision_ensemble)

                0
count  100.000000
mean     0.976644
std      0.021128
min      0.910448
25%      0.966667
50%      0.983051
75%      1.000000
max      1.000000
                0
count  100.000000
mean     0.908405
std      0.035110
min      0.833333
25%      0.884295
50%      0.913037
75%      0.932203
max      0.981818
                0
count  100.000000
mean     0.964841
std      0.025180
min      0.887324
25%      0.953125
50%      0.967742
75%      0.983871
max      1.000000


(None, None, None)

In [28]:
show_statistics(recall_spectrals), show_statistics(recall_spatials), show_statistics(recall_ensemble)

                0
count  100.000000
mean     0.936508
std      0.033004
min      0.841270
25%      0.920635
50%      0.936508
75%      0.968254
max      1.000000
                0
count  100.000000
mean     0.786349
std      0.050771
min      0.634921
25%      0.757937
50%      0.793651
75%      0.813492
max      0.888889
                0
count  100.000000
mean     0.967143
std      0.023365
min      0.888889
25%      0.952381
50%      0.968254
75%      0.984127
max      1.000000


(None, None, None)

In [29]:
show_statistics(accuracy_spectrals), show_statistics(accuracy_spatials), show_statistics(accuracy_ensemble)

                0
count  100.000000
mean     0.956746
std      0.017380
min      0.920635
25%      0.944444
50%      0.952381
75%      0.968254
max      0.992063
                0
count  100.000000
mean     0.852937
std      0.029209
min      0.785714
25%      0.833333
50%      0.849206
75%      0.873016
max      0.920635
                0
count  100.000000
mean     0.965556
std      0.016120
min      0.928571
25%      0.952381
50%      0.968254
75%      0.976190
max      1.000000


(None, None, None)

In [ ]:
show_statistics(all_score_spectrals_train, reshape=1), show_statistics(all_score_spatials_train, reshape=1), show_statistics(all_bias, reshape=1)

In [120]:
def gather_data(all_y, all_score_spectral_norms, all_score_spatial_norms, all_pred_spectral_norms, all_pred_spatial_norms):
    all_score_spectral_norms=np.array(all_score_spectral_norms).squeeze().reshape(-1)
    all_score_spatial_norms=np.array(all_score_spatial_norms).squeeze().reshape(-1)
    all_pred_spectral_norms=np.array(all_pred_spectral_norms).squeeze().reshape(-1)
    all_pred_spatial_norms=np.array(all_pred_spatial_norms).squeeze().reshape(-1)
    all_labels = []
    for xxx in all_y:
        all_labels += xxx.values.tolist()
    #show_statistics(all_score_spectral_norms), show_statistics(all_score_spatial_norms)
    return all_labels, all_score_spectral_norms, all_score_spatial_norms, all_pred_spectral_norms, all_pred_spatial_norms

In [225]:
def result_statistics(all_labels, all_pred_spectral_norms, all_pred_spatial_norms, all_score_spectral_norms, all_score_spatial_norms):
    both_right_num=0
    only_sprectral_right_num=0
    only_spatial_right_num=0
    both_wrong=0
    tot_num = len(all_labels)
    print('tot_num:', tot_num)
    only_spectral_right_pairs = []
    only_spatial_right_pairs = []
    for i_sample in range(tot_num):
        if all_labels[i_sample]==all_pred_spectral_norms[i_sample] and all_labels[i_sample]==all_pred_spatial_norms[i_sample]:
            both_right_num+=1
        elif all_labels[i_sample]==all_pred_spectral_norms[i_sample] and all_labels[i_sample]!=all_pred_spatial_norms[i_sample]:
            only_sprectral_right_num+=1
            only_spectral_right_pairs.append((all_score_spectral_norms[i_sample], all_score_spatial_norms[i_sample]))
        elif all_labels[i_sample]!=all_pred_spectral_norms[i_sample] and all_labels[i_sample]==all_pred_spatial_norms[i_sample]:
            only_spatial_right_num+=1
            only_spatial_right_pairs.append((all_score_spectral_norms[i_sample], all_score_spatial_norms[i_sample]))
        elif all_labels[i_sample]!=all_pred_spectral_norms[i_sample] and all_labels[i_sample]!=all_pred_spatial_norms[i_sample]:
            both_wrong+=1
        else:
            assert True
    print("both_right ratio:", both_right_num/tot_num)
    print("only_sprectral_right ratio:", only_sprectral_right_num/tot_num)
    print("only_spatial_right ratio:", only_spatial_right_num/tot_num)
    print("both_wrong ratio:", both_wrong/tot_num)
    return only_spectral_right_pairs, only_spatial_right_pairs

In [130]:
all_labels, all_score_spectral_norms, all_score_spatial_norms, all_pred_spectral_norms, all_pred_spatial_norms=gather_data(all_y_train, all_score_spectral_norms_train, all_score_spatial_norms_train, all_pred_spectral_norms_train, all_pred_spatial_norms_train)
only_spectral_right_pairs, only_spatial_right_pairs=result_statistics(all_labels, all_pred_spectral_norms, all_pred_spatial_norms, all_score_spectral_norms, all_score_spatial_norms)

(29400,) 29400
tot_num: 29400
both_right ratio: 0.9168707482993197
only_sprectral_right ratio: 0.08312925170068028
only_spatial_right ratio: 0.0
both_wrong ratio: 0.0


In [136]:
all_labels, all_score_spectral_norms, all_score_spatial_norms, all_pred_spectral_norms, all_pred_spatial_norms=gather_data(all_y_test, all_score_spectral_norms_test, all_score_spatial_norms_test, all_pred_spectral_norms_test, all_pred_spatial_norms_test)
only_spectral_right_pairs, only_spatial_right_pairs=result_statistics(all_labels, all_pred_spectral_norms, all_pred_spatial_norms, all_score_spectral_norms, all_score_spatial_norms)

tot_num: 12600
both_right ratio: 0.846984126984127
only_sprectral_right ratio: 0.11428571428571428
only_spatial_right ratio: 0.02642857142857143
both_wrong ratio: 0.012301587301587301


In [132]:
import matplotlib
matplotlib.use('TkAgg')
#%matplotlib inline
import matplotlib.pyplot as plt
tmp_list=[]
tmp_list1=[]
for pair in only_spectral_right_pairs:
    tmp_list.append(pair[0])
    tmp_list1.append(pair[1])
plt.figure()
plt.plot(tmp_list, tmp_list1, 'g.')
plt.xlim([-3,3])
plt.ylim([-3,3])
tmp_list=[]
tmp_list1=[]
for pair in only_spatial_right_pairs:
    tmp_list.append(pair[0])
    tmp_list1.append(pair[1])
plt.figure()
plt.plot(tmp_list, tmp_list1, 'r.')
plt.xlim([-3,3])
plt.ylim([-3,3])
plt.show(block=True)

In [137]:
import matplotlib
matplotlib.use('TkAgg')
#%matplotlib inline
import matplotlib.pyplot as plt

tmp_list = []
tmp_list1 = []
# only_sprectral_right_pairs only_spatial_right_pairs

#init_bias=0.01
init_bias=-1.0
bias = init_bias
step = 0.01
last_ratio = 0
last_ratio1 = 0
last_choose_num = 0
last_choose_num1 = 0
best_sum = 0

def judge_func(pair, bias):
    #yyy = abs(pair[0])+bias-abs(pair[1])

    if pair[1] > 0:
        yyy = abs(pair[0])+bias-abs(pair[1])
    else:
        yyy = 1

    #yyy = abs(pair[0])/abs(pair[1])-bias

    #res = svc_model.predict([[pair[0], pair[1]]])
    #yyy = 1 if res == 0 else -1
    return yyy

#if True:
while True:
    choose_num = 0 
    for pair in only_spectral_right_pairs:
        yyy = judge_func(pair, bias)
        if yyy >= 0:
            choose_num+=1

        #score_spectral_norm = pair[0]
        #score_spatial_norm = pair[1]
        #spectral_value = ((score_spectral_norm/3.0)+1.0)/2.0
        #spatial_value = ((score_spatial_norm/3.0)+1.0)/2.0
        #yyy = fuzzy_logic_infer(spectral_value, spatial_value, bias)
        #if np.sign(pair[0]) == np.sign(yyy):
        #    choose_num+=1

    ratio = choose_num/len(only_spectral_right_pairs)

    choose_num1 = 0 
    for pair in only_spatial_right_pairs:
        yyy = judge_func(pair, bias)
        if yyy < 0:
            choose_num1+=1

        #score_spectral_norm = pair[0]
        #score_spatial_norm = pair[1]
        #spectral_value = ((score_spectral_norm/3.0)+1.0)/2.0
        #spatial_value = ((score_spatial_norm/3.0)+1.0)/2.0
        #yyy = fuzzy_logic_infer(spectral_value, spatial_value, bias)
        #if np.sign(pair[1]) == np.sign(yyy):
        #    choose_num+=1

    ratio1 = choose_num1/len(only_spatial_right_pairs)
    if choose_num + choose_num1 > best_sum:
        bias += step
        best_bias = bias
        best_sum = choose_num + choose_num1
    else:
        bias += step
        #best_bias = bias
        #break
    if bias >= 2.0:
        break
    last_ratio = ratio
    last_ratio1 = ratio1
    last_choose_num = choose_num
    last_choose_num1 = choose_num1
    #print(bias, ratio, ratio1, choose_num, choose_num1, choose_num+choose_num1)
print("best_bias:", best_bias)
#print("svm model:")
print("fuzzy_logic_infer:")
print("ratio:", ratio, "ratio1:", ratio1)
print("total info:", choose_num+choose_num1, (choose_num+choose_num1)/(len(only_spectral_right_pairs)+len(only_spatial_right_pairs)))

best_bias: 0.16000000000000075
fuzzy_logic_infer:
ratio: 1.0 ratio1: 0.024024024024024024
total info: 1448 0.8166948674562888


In [19]:
import matplotlib
matplotlib.use('TkAgg')
#%matplotlib inline
import matplotlib.pyplot as plt
show_num = 1
y_show = all_y_tests[show_num].values*2 - 1
plt.plot(np.arange(len(all_score_spectral_norms[show_num])), all_score_spectral_norms[show_num], 'g.')
plt.plot(np.arange(len(all_score_spectral_norms[show_num])), all_score_spatial_norms[show_num], 'r')
plt.plot(np.arange(len(all_score_spectral_norms[show_num])), y_show, '*')
plt.show(block=True)

### train a model for ensemble by spectral/spatial accuracies

In [ ]:
"""
change seed = np.random.seed(10) to generate training data
"""
all_score_spectral_norms=np.array(all_score_spectral_norms).squeeze().reshape(-1, 1)
all_score_spatial_norms=np.array(all_score_spatial_norms).squeeze().reshape(-1, 1)
train_data = np.concatenate([all_score_spectral_norms, all_score_spatial_norms], 1)
train_label = []
for xxx in all_y_tests:
    train_label += xxx.values.tolist()
len(train_label), train_data.shape

In [91]:
train_data=[]
train_label = []
for pair in only_sprectral_right_pairs:
    train_data.append([pair[0], pair[1]])
    train_label.append(0)

for pair in only_spatial_right_pairs:
    train_data.append([pair[0], pair[1]])
    train_label.append(1)
train_data = np.array(train_data)
len(train_label), train_data.shape

(1887, (1887, 2))

In [95]:
from sklearn.svm import LinearSVC
#squared_hinge hinge False
svc_model = LinearSVC(C=10.0, random_state=42, penalty='l2', loss='hinge', dual=True)
svc_model.fit(train_data, train_label)
svc_model.score(train_data, train_label)

f:\Software\Anaconda\envs\pytorch\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.8791732909379968

In [80]:
train_data[0], svc_model.predict([train_data[0]]), train_label[0]

(array([-1.3042,  0.0054]), array([0]), 0)

In [153]:
import numpy as np

LDA = LinearDiscriminantAnalysis()

cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=2, random_state=42)

grid = dict()
grid['solver'] = ['eigen']
shrinkages = np.linspace(0.1, 0.9, 10).tolist()
#shrinkages = np.linspace(0.62, 0.64, 10).tolist()
shrinkages = np.linspace(0.01, 0.1, 10).tolist()
#shrinkages = np.linspace(0.001, 0.01, 10).tolist()
shrinkages = np.linspace(0.000001, 0.00001, 10).tolist()

grid['shrinkage'] = ['auto'] + shrinkages
#search = GridSearchCV(LDA, grid, scoring='precision', cv=cv, n_jobs=-1)
search = GridSearchCV(LDA, grid, scoring='accuracy', cv=cv, n_jobs=-1)

# only_spatial_features  only_spatial_features_norm
# X_with_spatial_features X_with_spatial_features_norm
# X_norm_with_spatial_features_norm X_norm
# only_spatial_features_norm_ave
# X
used_feature = train_data
results = search.fit(used_feature, train_label)
print('reault: %.3f' % results.best_score_)
print('Configuration:',results.best_params_)
print(used_feature.shape)

reault: 0.966
Configuration: {'shrinkage': 'auto', 'solver': 'eigen'}
(12600, 2)


f:\Software\Anaconda\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [161]:
show_statistics(accuracy_ensemble)

               0
count  10.000000
mean    0.972222
std     0.023139
min     0.944444
25%     0.948413
50%     0.976190
75%     0.994048
max     1.000000


### T-test

In [30]:
import scipy.stats as stats
stats.ttest_rel(accuracy_spectrals, accuracy_spatials)

TtestResult(statistic=30.92230963449535, pvalue=1.1299642198011607e-52, df=99)

In [31]:
stats.ttest_rel(accuracy_spectrals, accuracy_ensemble)

TtestResult(statistic=-6.650454379738139, pvalue=1.6200024805567017e-09, df=99)

In [32]:
stats.ttest_ind(accuracy_spectrals, accuracy_ensemble)

Ttest_indResult(statistic=-3.7163234396153975, pvalue=0.00026298441219991126)

In [33]:
stats.ttest_rel(accuracy_spatials, accuracy_ensemble)

TtestResult(statistic=-37.635072960917725, pvalue=1.8418585152070884e-60, df=99)

In [84]:
from scipy.stats import f_oneway
f_oneway(accuracy_spectrals, accuracy_spatials, accuracy_ensemble)

F_onewayResult(statistic=836.0034700315449, pvalue=1.020038496149818e-122)

In [76]:
df = pd.DataFrame([accuracy_spectrals, accuracy_spatials, accuracy_ensemble])
df = df.T
df

,0,1,2
0,0.960317,0.888889,0.960317
1,0.968254,0.849206,0.976190
2,0.976190,0.880952,0.984127
3,0.928571,0.880952,0.944444
4,0.968254,0.888889,0.984127
...,...,...,...
95,0.968254,0.880952,0.976190
96,0.952381,0.888889,0.968254
97,0.936508,0.849206,0.968254
98,0.928571,0.849206,0.936508


In [ ]:
import numpy as np 
import scipy.stats as stats
import matplotlib.pyplot as plt

for i_data in range(df.shape[1]):
    stats.probplot(df[i_data], dist="norm", plot=plt)
    plt.title("Probability Plot - " +  str(i_data))
    plt.show()

In [77]:
ratio = df.std(0).max() / df.std(0).min()
ratio

1.6255747139657795

In [77]:
df.std(0)**2

0    0.000367
1    0.000739
2    0.000265
dtype: float64

In [78]:
# Create ANOVA backbone table
data = [['Between Groups', '', '', '', '', '', ''], ['Within Groups', '', '', '', '', '', ''], ['Total', '', '', '', '', '', '']] 
anova_table = pd.DataFrame(data, columns = ['Source of Variation', 'SS', 'df', 'MS', 'F', 'P-value', 'F crit']) 
anova_table.set_index('Source of Variation', inplace = True)

num_sample = df.shape[0]
num_cls = df.shape[1]
# calculate SSTR and update anova table
SSTR = num_cls * (df.mean(0) - df.mean().mean())**2
anova_table['SS']['Between Groups'] = SSTR.sum()

# calculate SSE and update anova table
#tmp = 0
#for i_data in range(len(data)):
#    tmp += (data.iloc[i_data]- data.iloc[i_data].std())**2
#SSE = (data.count() - 1) * tmp.sum()

SSE = (num_cls - 1) * df.std(0)**2
anova_table['SS']['Within Groups'] = SSE.sum()

# calculate SSTR and update anova table
SSTR = SSTR.sum() + SSE.sum()
anova_table['SS']['Total'] = SSTR

# update degree of freedom
anova_table['df']['Between Groups'] = num_cls - 1
anova_table['df']['Within Groups'] = num_sample - num_cls
anova_table['df']['Total'] = num_sample - 1

# calculate MS
anova_table['MS'] = anova_table['SS'] / anova_table['df']

# calculate F 
F = anova_table['MS']['Between Groups'] / anova_table['MS']['Within Groups']
anova_table['F']['Between Groups'] = F

# p-value
anova_table['P-value']['Between Groups'] = 1 - stats.f.cdf(F, anova_table['df']['Between Groups'], anova_table['df']['Within Groups'])

# F critical 
alpha = 0.05
# possible types "right-tailed, left-tailed, two-tailed"
tail_hypothesis_type = "two-tailed"
if tail_hypothesis_type == "two-tailed":
    alpha /= 2
anova_table['F crit']['Between Groups'] = stats.f.ppf(1-alpha, anova_table['df']['Between Groups'], anova_table['df']['Within Groups'])

# Final ANOVA Table
anova_table

,SS,df,MS,F,P-value,F crit
Source of Variation,,,,,,
Between Groups,0.019222,2,0.009611,405.461683,0.0,3.832792
Within Groups,0.002299,97,0.000024,,,
Total,0.021521,99,0.000217,,,


In [79]:
#LSD = t.025, DFw * √MSW(1/n1 + 1/n1)
LSD = 2.326 * (0.000024*(1/df.shape[0] + 1/df.shape[0]))**0.5
LSD

0.0016115000713620834

In [80]:
abs(df[0].mean() - df[1].mean())

0.09333333333333316

In [81]:
abs(df[1].mean() - df[2].mean())

0.10214285714285687

In [82]:
abs(df[0].mean() - df[2].mean())

0.008809523809523712

In [83]:
# The p-value approach
print("Approach 1: The p-value approach to hypothesis testing in the decision rule")
conclusion = "Failed to reject the null hypothesis."
if anova_table['P-value']['Between Groups'] <= alpha:
    conclusion = "Null Hypothesis is rejected."
print("F-score is:", anova_table['F']['Between Groups'], " and p value is:", anova_table['P-value']['Between Groups'])    
print(conclusion)
    
# The critical value approach
print("\n--------------------------------------------------------------------------------------")
print("Approach 2: The critical value approach to hypothesis testing in the decision rule")
conclusion = "Failed to reject the null hypothesis."
if anova_table['F']['Between Groups'] > anova_table['F crit']['Between Groups']:
    conclusion = "Null Hypothesis is rejected."
print("F-score is:", anova_table['F']['Between Groups'], " and critical value is:", anova_table['F crit']['Between Groups'])
print(conclusion)

Approach 1: The p-value approach to hypothesis testing in the decision rule
F-score is: 405.4616829652971  and p value is: 1.1102230246251565e-16
Null Hypothesis is rejected.

--------------------------------------------------------------------------------------
Approach 2: The critical value approach to hypothesis testing in the decision rule
F-score is: 405.4616829652971  and critical value is: 3.8327917620793706
Null Hypothesis is rejected.


In [44]:
show_statistics(accuracy_spectrals)
show_statistics(accuracy_spatials)
show_statistics(accuracy_ensemble)

                 0
count  1000.000000
mean      0.961357
std       0.016712
min       0.904762
25%       0.952381
50%       0.960317
75%       0.976190
max       1.000000
                 0
count  1000.000000
mean      0.863659
std       0.026149
min       0.777778
25%       0.849206
50%       0.865079
75%       0.880952
max       0.936508
                 0
count  1000.000000
mean      0.968246
std       0.014980
min       0.920635
25%       0.960317
50%       0.968254
75%       0.976190
max       1.000000


In [49]:
valid_num = 0
bad_cases = []
good_cases = []
for i_acc, final_accuracy in enumerate(accuracy_ensemble):
    if final_accuracy >= max(accuracy_spectrals[i_acc], accuracy_spatials[i_acc]):
        valid_num += 1
        gap = final_accuracy - max(accuracy_spectrals[i_acc], accuracy_spatials[i_acc])
        if gap > 0.01:
            good_cases.append({'f': final_accuracy, 'sprectral': accuracy_spectrals[i_acc], 'spatial': accuracy_spatials[i_acc]})
    else:
        gap = max(accuracy_spectrals[i_acc], accuracy_spatials[i_acc]) - final_accuracy
        if gap > 0.01:
            bad_cases.append({'f': final_accuracy, 'sprectral': accuracy_spectrals[i_acc], 'spatial': accuracy_spatials[i_acc]})
valid_num / len(accuracy_ensemble), len(good_cases) / len(accuracy_ensemble), len(bad_cases) / len(accuracy_ensemble)

(0.959, 0.224, 0.001)

In [50]:
bad_cases

[{'f': 0.9523809523809523,
  'sprectral': 0.9682539682539683,
  'spatial': 0.9047619047619048}]

In [51]:
0.9682539682539683-0.9523809523809523
#0.9761904761904762-0.9682539682539683
#0.9523809523809523-0.9444444444444444

0.015873015873015928

## LDA from scratch

In [ ]:
from sklearn.preprocessing import LabelEncoder
df = pd.concat([X, label], axis= 1)
xxx = dict(list(df.groupby('label')))
yyy = xxx[0].mean(axis=0)
yyy.drop(['label'])

In [38]:
from sklearn.preprocessing import LabelEncoder
df = pd.concat([X, label], axis= 1)

class_feature_means = pd.DataFrame(columns=[0, 1])
for cls, rows in df.groupby('label'):
    class_feature_means[cls] = rows.mean(axis=0)
class_feature_means = class_feature_means.drop(['label'])
class_feature_means

,0,1
0,0.125274,0.112998
1,0.124180,0.111487
2,0.122471,0.109218
3,0.120787,0.107771
4,0.120364,0.107212
...,...,...
457,0.270462,0.237536
458,0.270214,0.236737
459,0.276498,0.243368
460,0.277633,0.243970


In [37]:
dim = len(class_feature_means)
dim

462

In [ ]:
within_class_scatter_matrix = np.zeros((dim,dim))
for c, rows in df.groupby('label'):
    rows = rows.drop(['label'], axis=1)
    s = np.zeros((dim,dim))
    for index, row in rows.iterrows():
        x  = row.values.reshape(dim, 1)
        mc  = class_feature_means[c].values.reshape(dim,1)
        s += (x - mc).dot((x - mc).T)
        within_class_scatter_matrix += s
within_class_scatter_matrix_show = pd.DataFrame(within_class_scatter_matrix)
within_class_scatter_matrix_show.describe()

In [ ]:
feature_means = df.mean().drop(['label'])
between_class_scatter_matrix = np.zeros((dim,dim))
for c in class_feature_means:    
    n = len(df.loc[df['label'] == c].index)
    
    mc, m = class_feature_means[c].values.reshape(dim,1), feature_means.values.reshape(dim,1)
    
    between_class_scatter_matrix += n * (mc - m).dot((mc - m).T)

between_class_scatter_matrix_show = pd.DataFrame(between_class_scatter_matrix)
between_class_scatter_matrix_show.describe()

In [44]:
eigen_values, eigen_vectors = np.linalg.eig(np.linalg.inv(within_class_scatter_matrix).dot(between_class_scatter_matrix))

In [ ]:
pairs = [(np.abs(eigen_values[i]), eigen_vectors[:,i]) for i in range(len(eigen_values))]
pairs = sorted(pairs, key=lambda x: x[0], reverse=True)
for pair in pairs:
    print(pair[0])

In [ ]:
eigen_value_sums = sum(eigen_values)
print('Explained Variance')
for i, pair in enumerate(pairs):
    print('Eigenvector {}: {}'.format(i, (pair[0]/eigen_value_sums).real))

In [48]:
w_matrix = np.hstack((pairs[0][1].reshape(dim,1), pairs[1][1].reshape(dim,1))).real

In [49]:
w_matrix.shape

(462, 2)

In [50]:
X_lda = np.array(X.dot(w_matrix))

In [55]:
X_lda.shape

(210, 2)

In [ ]:
if X_lda.shape[1] >= 2:
    plt.xlabel('LD1')
    plt.ylabel('LD2')
    plt.scatter(
        X_lda[:,0],
        X_lda[:,1],
        c=y,
        cmap='rainbow',
        alpha=0.5,
        edgecolors='b'
    )
    plt.show(block=True)
print(X_lda[:5])

# PLS-DA

In [39]:
from sys import stdout
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.signal import savgol_filter

from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score

In [51]:
import matplotlib
matplotlib.use('TkAgg')
wl = np.linspace(393, 1011, x.shape[1])
with plt.style.context('ggplot'):
    plt.plot(wl, x.T)
    plt.xlabel("Wavelengths (nm)")
    plt.ylabel("Reflectance")
    plt.show(block=True)
#X2 = savgol_filter(x, 17, polyorder=2, deriv=2)

#plt.figure(figsize=(8, 4.5))
#with plt.style.context('ggplot'):
#    plt.plot(wl, X2.T)
#    plt.xlabel("Wavelengths (nm)")
#    plt.ylabel("D2 Absorbance")
#    plt.show()

In [266]:
def optimise_pls_cv(X, y, n_comp):
    # Define PLS object
    pls = PLSRegression(n_components=n_comp)

    # Cross-validation
    y_cv = cross_val_predict(pls, X, y, cv=10)

    # Calculate scores
    r2 = r2_score(y, y_cv)
    mse = mean_squared_error(y, y_cv)
    rpd = y.std()/np.sqrt(mse)
    
    return (y_cv, r2, mse, rpd)

In [267]:
# test with 40 components
from tqdm import tqdm
r2s = []
mses = []
rpds = []
xticks = np.arange(1, 41)
for n_comp in tqdm(xticks):
    y_cv, r2, mse, rpd = optimise_pls_cv(x, y, n_comp)
    r2s.append(r2)
    mses.append(mse)
    rpds.append(rpd)

100%|██████████| 40/40 [00:27<00:00,  1.45it/s]


In [268]:
# Plot the mses
def plot_metrics(vals, ylabel, objective):
    with plt.style.context('ggplot'):
        plt.plot(xticks, np.array(vals), '-v', color='blue', mfc='blue')
        if objective=='min':
            idx = np.argmin(vals)
        else:
            idx = np.argmax(vals)
        plt.plot(xticks[idx], np.array(vals)[idx], 'P', ms=10, mfc='red')

        plt.xlabel('Number of PLS components')
        plt.xticks = xticks
        plt.ylabel(ylabel)
        plt.title('PLS')
    plt.show()

plot_metrics(mses, 'MSE', 'min')
plot_metrics(rpds, 'RPD', 'max')
plot_metrics(r2s, 'R2', 'max')

In [269]:
#y_cv, r2, mse, rpd = optimise_pls_cv(X2, y, 6)
y_cv, r2, mse, rpd = optimise_pls_cv(x, y, 17)
print('R2: %0.4f, MSE: %0.4f, RPD: %0.4f' %(r2, mse, rpd))

plt.figure(figsize=(6, 6))
with plt.style.context('ggplot'):
    plt.scatter(y, y_cv, color='red')
    plt.plot(y, y, '-g', label='Expected regression line')
    z = np.polyfit(y, y_cv, 1)
    plt.plot(np.polyval(z, y), y, color='blue', label='Predicted regression line')
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.legend()
    plt.plot()
    plt.show()

R2: 0.4420, MSE: 0.1395, RPD: 1.3387


In [40]:
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix,recall_score,classification_report,accuracy_score,precision_score

def accuracy_component(xc, yc, xv, yv, component, n_fold, random_state):
    k_range = np.linspace(1, component,component)

    kf=KFold(n_splits=n_fold, random_state=random_state, shuffle=True)
    accuracy_validation=np.zeros((component,))
    accuracy_test=np.zeros((component,))
    precision_test=np.zeros((component,))
    recall_test=np.zeros((component,))

    best_p = 0
    best_r = 0
    best_a = 0
    for j in range(component):
        p=0
        acc=0
        model_pls=PLSRegression(n_components=j+1)
        
        yc_labels = pd.get_dummies(yc)
        
        model_pls.fit(xc,yc_labels)
        y_pred = model_pls.predict(xv)
        
        y_pred = np.array([np.argmax(i) for i in y_pred])

        accuracy_test[j]=accuracy_score(yv,y_pred)
        precision_test[j]=recall_score(yv,y_pred)
        recall_test[j]=precision_score(yv,y_pred)
        for train_index, test_index in kf.split(xc):
            X_train, X_test = xc[train_index], xc[test_index]
            y_train, y_test = yc[train_index], yc[test_index]
            
            YC_labels = pd.get_dummies(y_train)
            YV_labels=pd.get_dummies(y_test)
            model_1=PLSRegression(n_components=j+1)
            model_1.fit(X_train,YC_labels)
            Y_pred = model_1.predict(X_test)
            Y_pred = np.array([np.argmax(i1) for i1 in Y_pred])
            acc=accuracy_score(y_test,Y_pred)+acc
            p=p+1
        accuracy_validation[j]=acc/p
    #print(accuracy_validation)

    #plt.plot(k_range, accuracy_validation.T, 'o-',label="Cross-validation",color="r")
    #plt.plot(k_range, accuracy_test.T, 'o-',label="Test",color="b")
    #plt.xlabel("N components")
    #plt.ylabel("Accuracy")
    #plt.legend(loc="best")
    #plt.rc('font',family='Times New Roman')
    #plt.rcParams['font.size'] = 10
    #plt.show()
    precision_test=np.array(precision_test)
    recall_test=np.array(recall_test)
    accuracy_test=np.array(accuracy_test)
    idx = np.argmax(accuracy_test)
    print(idx+1, precision_test[idx], recall_test[idx], accuracy_test[idx])
    best_p = precision_test[idx]
    best_r = recall_test[idx]
    best_a = accuracy_test[idx]
    return best_p, best_r, best_a
#seed_int = 0
#np.random.seed(seed_int)
##np.random.seed(int(time.time()))
#random_state = np.random.randint(0, 1e4)
#print('random_state:', random_state)
#accuracy_component(X_train, y_train, X_test, y_test, component=30, n_fold=4, random_state=random_state)
#x=0

In [41]:
np.random.seed(0)
#np.random.seed(int(time.time()))
#random_state = np.random.randint(0, 1e4)
random_state=0
#X_=X.values
#X_norm_=X_norm.values
#label_=label.values

#X_norm
#spatial_features_norm
#X_norm_with_spatial_features_norm

#X_norm_with_spatial_features_norm_MRMR_norm
#X_norm_mrmr_with_spatial_features_norm_mrmr

#X_norm_with_spatial_features_norm_PCA_norm
X_norm_ = pd.DataFrame(X_norm_mrmr_with_spatial_features_norm_mrmr).values
label_ = pd.DataFrame(label).values

seed_int = 0
np.random.seed(seed_int)
#np.random.seed(int(time.time()))
random_state = 0
#random_state = np.random.randint(0, 1e4)
best_ps, best_rs, best_as = [], [], []
print('random_state:', random_state)
X_train, X_test, y_train, y_test = train_test_split(
    X_norm_, label_, test_size=0.30, random_state=random_state, shuffle=True, stratify=label)
y_train=y_train.squeeze()
y_test=y_test.squeeze()
component = min(X_norm_.shape[1], 35)
best_p, best_r, best_a = accuracy_component(X_train, y_train, X_test, y_test, component=component, n_fold=4, random_state=random_state)

random_state: 0
21 0.873015873015873 0.9322033898305084 0.9047619047619048


In [42]:
# 28 7 6
# 28 5 6 22 1
# 28 5 13 5 1
# 28 5 13 21 1
model_params={'n_components': 21}
np.random.seed(0)
seed=True
#X_norm
#spatial_features_norm
#X_norm_with_spatial_features_norm

#X_norm_with_spatial_features_norm_MRMR_norm
#X_norm_mrmr_with_spatial_features_norm_mrmr

#X_norm_with_spatial_features_norm_PCA_norm
X_norm_ = pd.DataFrame(X_norm_mrmr_with_spatial_features_norm_mrmr).values
label_ = pd.DataFrame(label).values
_ = repeated_tests(PLSRegression, X_norm_, label, split_ratio=0.3, test_num=100, seed=seed, model_params=model_params, one_hot_y=True)

100%|██████████| 100/100 [00:06<00:00, 14.54it/s]

start show results
(420, 69) (294, 69) (126, 69) 

precision:
                0
count  100.000000
mean     0.893216
std      0.036219
min      0.771930
25%      0.870370
50%      0.892857
75%      0.917500
max      0.963636 

recall:
                0
count  100.000000
mean     0.802540
std      0.047716
min      0.682540
25%      0.777778
50%      0.809524
75%      0.841270
max      0.904762 

accuracy:
                0
count  100.000000
mean     0.852619
std      0.028499
min      0.746032
25%      0.833333
50%      0.853175
75%      0.873016
max      0.912698


# PCA + DecisionTreeClassifier 

In [317]:
X = finalDf[['PC1', 'PC2', 'PC3']]
y = finalDf[['class']]

In [318]:
from sklearn.preprocessing import StandardScaler

x_norm = StandardScaler().fit_transform(X)
X = x_norm

In [319]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, shuffle=True, stratify=y)

In [327]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)
conf_mat, (conf_mat[0,0]+conf_mat[1,1])/conf_mat.sum()

array([[14,  7],
       [ 7, 25]], dtype=int64)

# PCA + LogisticRegression 

In [424]:
X = finalDf[['PC1', 'PC2', 'PC3']]
y = finalDf[['class']]

In [425]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, shuffle=True, stratify=y))

In [427]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(X_train)
# Apply transform to both the training set and the test set.
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [428]:
pca = PCA(.95)

In [429]:
pca.fit(X_train)

PCA(n_components=0.95)

In [433]:
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)
X_train.shape

(157, 3)

In [431]:
from sklearn.linear_model import LogisticRegression

In [432]:
# all parameters not specified are set to their defaults
# default solver is incredibly slow which is why it was changed to 'lbfgs'
logisticRegr = LogisticRegression(solver = 'lbfgs')

In [434]:
logisticRegr.fit(X_train, y_train)

f:\Software\Anaconda\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



LogisticRegression()

In [435]:
logisticRegr.predict(X_test[0].reshape(1,-1))

array([1])

In [436]:
logisticRegr.predict(X_test[0:10])

array([1, 1, 1, 1, 1, 0, 0, 1, 0, 1])

In [438]:
logisticRegr.score(X_test, y_test)

0.6792452830188679

# image features

In [ ]:
! pip install mahotas
# load Data section

In [278]:
sample_dirs = []
for dir_name in save_dir_im_data_bad.iterdir():
    sample_dirs.append(dir_name)
sample_dirs = sorted(sample_dirs, key=lambda x:int(x.name))
sample_idx=17
sample_dirs[sample_idx]

WindowsPath('D:/BoyangDeng/BlueberryClassification/datasets/BlueberryScansforDestructiveTesting06132022/BadBlueberryScans/im_data_local/18')

In [281]:
import mahotas
import cv2
path = (sample_dirs[sample_idx] / (str(100)+'_'+str(wavelengths[100])+'.png')).__str__()
im = cv2.imread(path)

im_color = cv2.applyColorMap(im, cv2.COLORMAP_JET )
cv2.namedWindow('x', 0)
cv2.imshow('x', im_color)
cv2.waitKey(1)
gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

#import plotly.express as px
#fig = px.imshow(im)
##fig.update_traces(showscale=False)
#fig.update_layout(yaxis={'visible': False, 'showticklabels': False},
#                  xaxis={'visible': False, 'showticklabels': False})
#fig.write_image(r"F:\test\show_spatial.png", width=1400, height=1000)

## haralick

In [ ]:
gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
features = mahotas.features.haralick(gray)#.mean(axis=0)
im.shape, features.shape
mean_feature=features.mean(axis=0)

import plotly.graph_objects as go
fig = go.Figure()				
fig.add_trace(go.Bar(
    y=mean_feature,
    x=np.arange(len(mean_feature)),
))
fig=plotly_fig_config(fig,title='Haralick Features')
fig.show()
#fig.write_image(os.path.join(tmp_save_dir,"show_spatial_haralick.png"), width=700, height=500)

In [276]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [282]:
options = {}
options['dharalick'] = 3
X  = eng.Bfx_haralick(gray, [], options)
X = np.asarray(X)
X.shape, X

((1, 28),
 array([[  0.2191,   0.2735, 143.5274,   0.2735,   0.8835,   3.9465,
           2.2357,   1.7108,   1.9167,   0.0744,   0.5841,  -0.5117,
           0.7312,   0.7942,   0.2259,   0.338 , 147.2086,   0.338 ,
           0.8933,   3.9641,   2.2676,   1.728 ,   1.9795,   0.0774,
           0.6454,   0.5518,   0.7596,   0.8299]]))

## Gabor

In [9]:
# We create our gabor filters, and then apply them to our image
from modeling_spatial_features import create_gaborfilter, apply_filter
import cv2
gfilters = create_gaborfilter()
image_g = apply_filter(im, gfilters)

im_color = cv2.applyColorMap(image_g, cv2.COLORMAP_JET )
cv2.namedWindow('x', 1)
cv2.imshow('x', im_color)
cv2.waitKey()

13

In [ ]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [38]:
options = {}
options['Lgabor'] = np.float32(8)
options['Sgabor'] = np.float32(8)
options['fhgabor'] = np.float32(2)
options['flgabor'] = np.float32(0.1)
options['Mgabor'] = np.float32(21)
options['show'] = 1
X  = eng.Bfx_gabor(gray, options)
X = np.asarray(X)
X.shape, X

((1, 67),
 array([[ 64.0406,  89.274 ,   3.5201,   3.7964,   5.6593,  15.7901,
          33.2222, 122.9141,  87.1522,  74.7226,   3.5489,   3.6135,
           5.3465,  14.8141,  34.9597, 107.5493,  95.2862,  59.3939,
          14.8206,   3.4314,   4.7465,   9.6991,  32.7083,  86.3802,
          87.1502,  74.7183,   3.781 ,   3.8601,   5.6948,  15.3251,
          35.3988, 110.6253,  64.0736,  89.2555,   3.8249,   4.2491,
           6.875 ,  16.7535,  33.619 , 126.7573,  87.1502,  74.7173,
           3.8164,   4.0252,   6.4123,  16.4903,  39.4573, 113.7675,
          95.2905,  59.4307,  14.8756,   3.5164,   5.2442,  11.0141,
          37.5938,  90.9629,  87.1521,  74.723 ,   3.546 ,   3.5923,
           5.3221,  15.0155,  38.0228, 110.0138, 126.7573,   3.4314,
          35.9408]]))

## LBP

In [ ]:
! pip install scikit-image

In [6]:
from modeling_spatial_features import LocalBinaryPatterns
import argparse
import cv2
import os
#desc = LocalBinaryPatterns(24, 8)
desc = LocalBinaryPatterns(24, 3)
data = []
labels = []

In [18]:
gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
hist = desc.describe(gray)
data.append(hist)
hist

array([0.0384, 0.0217, 0.0165, 0.0128, 0.0133, 0.014 , 0.0155, 0.0199,
       0.0237, 0.0295, 0.0363, 0.0395, 0.0417, 0.0404, 0.0266, 0.0228,
       0.0181, 0.0172, 0.0149, 0.0127, 0.0133, 0.0121, 0.0158, 0.02  ,
       0.0441, 0.4189])

In [41]:
import plotly.graph_objects as go

fig = go.Figure()				
fig.add_trace(go.Bar(
    y=hist,	
    x=np.arange(len(hist)),
))
fig=plotly_fig_config(fig,title='LBP Features')

fig.show()
fig.write_image(os.path.join(tmp_save_dir,"show_spatial_LBP.png"), width=700, height=500)

In [5]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [25]:
options = {}
options['vdiv'] = 1
options['hdiv'] = 1
options['semantic'] = 0
options['samples'] = np.float(8)
options['mappingtype'] = 'u2'
X = eng.Bfx_lbp(gray, [], options)
X = np.asarray(X)
X.shape

C:\Users\AFSALab\AppData\Local\Temp\ipykernel_3880\3222111.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  options['samples'] = np.float(8)


(1, 59)

## BSIF

In [11]:
from modeling_spatial_features import bsif

gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
img_bsif, hist = bsif(gray)

img_bsif=cv.normalize(img_bsif,  img_bsif, 0, 255, cv.NORM_MINMAX)
img_bsif=img_bsif.astype(np.uint8)
im_color = cv2.applyColorMap(img_bsif, cv2.COLORMAP_JET)
cv2.namedWindow('x', 1)
cv2.imshow('x', im_color)
cv2.waitKey()
#show images
#fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 8), sharex=True, sharey=True)
#plt.subplots_adjust(0.15)
#axes[0].imshow(gray, cmap="gray")
#axes[0].axis('off')
#axes[0].set_title('Original Image', fontsize=20)
#axes[1].imshow(img_bsif,cmap="gray")
#axes[1].axis('off')
#axes[1].set_title('BSIF Image', fontsize=20)
#fig.tight_layout()
#plt.show()

13

In [ ]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [ ]:
#failed
options = {}
options['vdiv'] = np.float32(1)
options['hdiv'] = np.float32(1)
options['filter'] = np.float32(7)
options['bits'] = np.float32(11)
options['mode'] = 'h'
X  = eng.Bfx_bsif(gray, [], options)
X = np.asarray(X)
X.shape, X

In [ ]:
import spatial_feature_BSIF
X = spatial_feature_BSIF.bsif(gray, mode='nh')
X.shape, X

## ICA

In [ ]:
from sklearn.decomposition import FastICA

gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

gray = gray.reshape(1, -1)
gray = np.concatenate([gray, gray])
"""
need many samples
"""
transformer = FastICA(n_components=7,
        random_state=0,
        whiten='unit-variance')
X_transformed = transformer.fit_transform(gray)
X_transformed.shape

## Hu-Moments

In [20]:
#skimage.measure.moments_hu 
gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
#_,gray_t = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
M = cv2.moments(gray)
huMoments = cv2.HuMoments(M)
huMoments
#xxx=huMoments.squeeze()
#import plotly.graph_objects as go
#fig = go.Figure()				
#fig.add_trace(go.Bar(
#    y=xxx,
#    x=np.arange(len(xxx)),
#))
#fig=plotly_fig_config(fig,title='Hu-Moments Features')
#fig.show()
#fig.write_image(os.path.join(tmp_save_dir,"show_spatial_huMoments.png"), width=700, height=500)

array([[ 0.0036],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [-0.    ],
       [ 0.    ]])

In [19]:
for i in range(0,7):
   huMoments[i] = -1* np.copysign(1.0, huMoments[i]) * np.log10(abs(huMoments[i]))
huMoments

array([[  2.4447],
       [  5.8302],
       [  8.3021],
       [  8.8751],
       [ 17.6909],
       [-11.9522],
       [ 17.5577]])

Note that hu[0] is not comparable in magnitude as hu[6]. We can use use a log transform given below to bring them in the same range

central moments are translation invariant but not scale and rotation invariant  
normalized central moments are translation and scale invariant but not rotation invariant

In [14]:
from skimage.measure import moments_hu, moments_normalized, moments_central
gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
mu = moments_central(gray)
nu = moments_normalized(mu)
moments_hu(nu)

array([ 0.0036,  0.    ,  0.    ,  0.    ,  0.    , -0.    , -0.    ])

# feature selection

## load data

In [1]:
from modeling_spectral_data import load_spectral_mean_data_xy
x, y = load_spectral_mean_data_xy()

c:\Users\AFSALab\.conda\envs\pytorch\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\AFSALab\.conda\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
c:\Users\AFSALab\.conda\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\AFSALab\.conda\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [ ]:
X =  pd.DataFrame(x)
label = pd.DataFrame(y, columns=['label'])

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=0.30, random_state=42)
X_train.shape

(147, 462)

## SBS

In [17]:
from sklearn.svm import LinearSVC
model = LinearSVC(C=10.0, random_state=42)
#model.fit(X_train, y_train)
#model.score(X_test, y_test)

In [ ]:
"""
high time consumption
10 featrure, 3 min
128 featrure, 62min
"""
from sklearn.feature_selection import SequentialFeatureSelector
n_features_to_select = 128
sfs = SequentialFeatureSelector(model, n_features_to_select=n_features_to_select)
sfs.fit(X_train, y_train)

In [ ]:
sfs.get_support()

In [21]:
X_train_sfs = sfs.transform(X_train)
X_test_sfs = sfs.transform(X_test)
X_train_sfs.shape

(147, 128)

In [22]:
from sklearn.svm import LinearSVC
model = LinearSVC(C=10.0, random_state=42)
model.fit(X_train_sfs, y_train)
model.score(X_test_sfs, y_test)

f:\Software\Anaconda\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
f:\Software\Anaconda\envs\pytorch\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.8095238095238095

## MRMR
minimum redundancy maximum relevance (MRMR)

In [49]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [95]:
"""
X_norm
spatial_features_norm
X_norm_with_spatial_features_norm
"""
#eng.fsrmrmr(X_norm, label.values.astype(np.float32))
idx_matlab = eng.fscmrmr(X_norm, label.values)
idx_matlab = np.array(idx_matlab)
feature_num = 303
idx_spectral = idx_matlab[0, :feature_num]-1

idx_matlab = eng.fscmrmr(spatial_features_norm, label.values)
idx_matlab = np.array(idx_matlab)
feature_num = 5
idx_spatial = idx_matlab[0, :feature_num]-1

In [97]:
X_norm_MRMR = X_norm[:, idx_spectral.astype(int)]
spatial_features_norm_MRMR = spatial_features_norm[:, idx_spatial.astype(int)]
X_norm_with_spatial_features_norm_MRMR = np.concatenate([X_norm_MRMR, spatial_features_norm_MRMR], 1)
X_norm_with_spatial_features_norm_MRMR_norm = X_norm_with_spatial_features_norm_MRMR
X_norm_with_spatial_features_norm_MRMR.shape

(420, 308)

In [99]:
path_X_norm_MRMR = r'D:\BoyangDeng\BlueberryClassification\datasets\mean_data_python\X_norm_MRMR.npy'
np.save(path_X_norm_MRMR, X_norm_MRMR)
path_spatial_features_norm_MRMR = r'D:\BoyangDeng\BlueberryClassification\datasets\mean_data_python\spatial_features_norm_MRMR.npy'
np.save(path_spatial_features_norm_MRMR, spatial_features_norm_MRMR)
X_norm_MRMR.shape, spatial_features_norm_MRMR.shape

((420, 303), (420, 5))

In [100]:
X_norm_MRMR = np.load(path_X_norm_MRMR, allow_pickle='TRUE')
spatial_features_norm_MRMR = np.load(path_spatial_features_norm_MRMR, allow_pickle='TRUE')
X_norm_MRMR.shape, spatial_features_norm_MRMR.shape

((420, 303), (420, 5))

## PCA

In [25]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.8)
X_norm_with_spatial_features_norm_PCA = pca.fit_transform(X_norm_with_spatial_features_norm)
#X_norm_with_spatial_features_norm_PCA = X_norm_with_spatial_features_norm_PCA[:, pca.explained_variance_ratio_>0.01]
X_norm_with_spatial_features_norm_PCA_norm = (X_norm_with_spatial_features_norm_PCA - X_norm_with_spatial_features_norm_PCA.mean(0)) / X_norm_with_spatial_features_norm_PCA.std(0)
X_norm_with_spatial_features_norm_PCA_norm.shape

(420, 28)

In [78]:
pca.explained_variance_ratio_

array([0.3029, 0.1419, 0.0953, 0.0708, 0.0304, 0.0211, 0.017 , 0.0149,
       0.0117, 0.0097, 0.0089, 0.008 , 0.0079, 0.0072, 0.0067, 0.006 ,
       0.0052, 0.0045, 0.0041, 0.0039, 0.0036, 0.0034, 0.0032, 0.0031,
       0.003 , 0.0029, 0.0027, 0.0027])

# pytorch model

## basic pipline

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [481]:
selected_bands = [460, 600, 690, 750, 840, 930]
target_values = []
target_idxes = []
for target in selected_bands:
    nearest_value, nearest_idx= find_nearest(wavelengths, target)
    target_values.append(nearest_value)
    target_idxes.append(nearest_idx)

([459.6, 599.97, 690.66, 749.6, 839.76, 930.39], [50, 156, 224, 268, 335, 402])

In [ ]:
save_dir_origin_data_good
save_dir_im_data_good

save_dir_origin_data_bad
save_dir_im_data_bad

In [484]:
# good_1_42 good_43_84 good_dir
# bad_1_42 bad_43_84 bad_85_126 bad_dir
save_path_good = good_dir / 'good_1_42.npy'.__str__()
save_path_good1 = good_dir / 'good_43_84.npy'.__str__()
save_path_bad = bad_dir / 'bad_1_42.npy'.__str__()
save_path_bad1 = bad_dir / 'bad_43_84.npy'.__str__()
save_path_bad2 = bad_dir / 'bad_85_126.npy'.__str__()

In [489]:
hyper_dict_good = np.load(save_path_good, allow_pickle='TRUE').item()
hyper_dict_good1 = np.load(save_path_good1, allow_pickle='TRUE').item()
hyper_dict_bad = np.load(save_path_bad, allow_pickle='TRUE').item()
hyper_dict_bad1 = np.load(save_path_bad1, allow_pickle='TRUE').item()
hyper_dict_bad2 = np.load(save_path_bad2, allow_pickle='TRUE').item()

In [493]:
hyper_dict_new = {}
for key in hyper_dict_good:
    hyper_dict_new[key] = hyper_dict_good[key]
for key in hyper_dict_good1:
    hyper_dict_new[key+42] = hyper_dict_good1[key]
hyper_dict_new.keys()
hyper_dict_good_tot = hyper_dict_new
len(hyper_dict_good_tot)

84

In [492]:
hyper_dict_new = {}
for key in hyper_dict_bad:
    hyper_dict_new[key] = hyper_dict_bad[key]
for key in hyper_dict_bad1:
    hyper_dict_new[key+42] = hyper_dict_bad1[key]
for key in hyper_dict_bad2:
    hyper_dict_new[key+84] = hyper_dict_bad2[key]
hyper_dict_new.keys()
hyper_dict_bad_tot = hyper_dict_new
len(hyper_dict_bad_tot)

126

In [562]:
from torch.utils.data import Dataset
from PIL import Image
import cv2
import os
 
class HyperData(Dataset):
    def __init__(self, root_dir, selected_bands, cls, hyper_dict_tot):
        self.root_dir = root_dir
        self.expand_size = (256, 256)

        target_idxes = []
        target_values = []
        for target in selected_bands:
            nearest_value, nearest_idx= find_nearest(wavelengths, target)
            target_idxes.append(nearest_idx)
            target_values.append(nearest_value)

        self.target_idxes = target_idxes
        self.target_values = target_values

        self.root_dir = root_dir
        self.cls = cls
        self.hyper_dict_tot = hyper_dict_tot

        sample_dirs = []
        for dir_name in root_dir.iterdir():
            sample_dirs.append(dir_name)
        sample_dirs = sorted(sample_dirs, key=lambda x:int(x.name))
        self.sample_dirs = sample_dirs
 
    def __getitem__(self, idx):
        sample_dir = self.sample_dirs[idx]
        imgs = []
        haralick_features = []
        for i_idx, _ in enumerate(self.target_idxes):
            path = (sample_dir / (str(self.target_idxes[i_idx])+'_'+str(self.target_values[i_idx])+'.png')).__str__()
            img = cv2.imread(path)
            img = img[:, :, 0]
            h, w = img.shape[:2]

            one_im_haralick_features = mahotas.features.haralick(img).mean(axis=0)

            expand_h, expand_w = self.expand_size
            img = np.pad(img, pad_width=[(0, expand_h-h),(0, expand_w-w)], mode='constant', constant_values=0)

            #fig = px.imshow(img)
            #fig.show()

            label = self.cls
            imgs.append(img)
            haralick_features.append(one_im_haralick_features)
        imgs = np.array(imgs)
        labels = np.array(label)
        spectral_features = self.hyper_dict_tot[idx]
        spectral_features = np.array(spectral_features)
        spatial_features = np.array(haralick_features).reshape(-1, 13*len(self.target_idxes)).squeeze()
        return imgs, labels, spectral_features, spatial_features
 
    def __len__(self):
        return len(self.sample_dirs)


In [563]:
good_dataset = HyperData(save_dir_im_data_good, selected_bands, 0, hyper_dict_good_tot)
bad_dataset = HyperData(save_dir_im_data_bad, selected_bands, 1, hyper_dict_bad_tot)
tot_dataset = good_dataset + bad_dataset

In [534]:
sample_num = len(tot_dataset)
train_num = int(sample_num * 0.7)
test_num = sample_num - train_num
train_num, test_num

(147, 63)

In [564]:
import torch
from torch.utils.data import random_split
training_data, test_data = random_split(tot_dataset, [train_num, test_num], generator=torch.Generator().manual_seed(42))

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 4

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y, spectral_features, spatial_features in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    print(f"Shape of spectral_features: {spectral_features.shape} {spectral_features.dtype}")
    print(f"Shape of spatial_features: {spatial_features.shape} {spatial_features.dtype}")

    print(y)
    ims = X[0].numpy()
    im_num = ims.shape[0]
    for i_im in range(im_num):
        fig = px.imshow(ims[i_im])
        fig.show()
    break

In [546]:
# Get cpu or gpu device for training.
from torch import nn

"""
model design
"""
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self, c_input, device, final_cls, fc_tot_feature_num):
        super().__init__()

        self.bias = torch.Tensor([128.0]).type(torch.float32).to(device)
        self.scale = torch.Tensor([255.0]).type(torch.float32).to(device)

        self.preprocess = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            #transforms.ToTensor(),
        ])

        self.flatten = nn.Flatten()

        c_input = c_input

        self.cnn1 = nn.Conv2d(c_input, 32, kernel_size=3, stride=2, padding=1)
        self.relu1 = nn.ReLU()

        self.cnn2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=0)
        self.relu2 = nn.ReLU()
        self.max_pool = nn.MaxPool2d(2, stride=2, padding=1)

        self.cnn3 = nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=0)
        self.relu3 = nn.ReLU()

        self.max_pool2 = nn.MaxPool2d(2, stride=2, padding=1)

        self.linear_relu_stack = nn.Sequential(
            #nn.Linear(4096, 512),
            nn.Linear(fc_tot_feature_num, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, final_cls)
        )

    def forward(self, x, spectral_features=None, spatial_features=None):
        x = self.preprocess(x)
        x = (x - self.bias)/self.scale
        x = self.cnn1(x)
        x = self.relu1(x)
        x = self.cnn2(x)
        x = self.relu2(x)
        x = self.max_pool(x)
        x = self.cnn3(x)
        x = self.relu3(x)
        x = self.max_pool2(x)
        x = self.flatten(x)
        if spectral_features is not None:
            x = torch.concat([x, spectral_features.type(torch.float32)], axis=1)
        if spatial_features is not None:
            x = torch.concat([x, spatial_features.type(torch.float32)], axis=1)
        logits = self.linear_relu_stack(x)
        return logits

c_input = len(selected_bands)
model = NeuralNetwork(c_input, device, final_cls=2, fc_tot_feature_num=4636).to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (cnn1): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu1): ReLU()
  (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu2): ReLU()
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=1152, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y, spectral_features, spatial_features) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        spectral_features, spatial_features = spectral_features.to(device), spatial_features.to(device)

        # Compute prediction error
        pred = model(X, spectral_features, spatial_features)
        loss = loss_fn(pred, y.type(torch.int64))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")



In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y, spectral_features, spatial_features in dataloader:
            X, y = X.to(device), y.to(device)
            spectral_features, spatial_features = spectral_features.to(device), spatial_features.to(device)

            pred = model(X, spectral_features, spatial_features)
            test_loss += loss_fn(pred, y.type(torch.int64)).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [ ]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

In [ ]:
classes = [
    "good",
    "bad",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

## try pretained resnet

In [ ]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model.eval()

In [ ]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
import torchvision
from tqdm import tqdm
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

path_to_data = '/home/llj/ld/data/OpenEdit/train_latest/images/input_image/'
BATCH_SIZE = 256
nThreads = 4
target_dir = 'targetdir'


test_data = torchvision.datasets.ImageFolder(path_to_data, preprocess)
image_names = test_data.samples
data_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE,)
model.to('cuda')

probability = []
count = 0
result = []

with torch.no_grad():
    for (x, y) in tqdm(data_loader, desc="Evaluating", leave=False):
        x = x.to('cuda')
        y = y.to('cuda')

        output = model(x)
        for index in range(output.shape[0]):
            if output[index,703]>0.5:
                result.append(image_names[count*BATCH_SIZE+index][0])
        count += 1
        # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
        # print(output[-1])
        # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
        # print(torch.nn.functional.softmax(output[-1], dim=0))

In [ ]:
import torchvision
resnet18 = torchvision.models.resnet18(pretrained=True)

# 修改全连接层的输出
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, 2)

# 加载模型参数
#checkpoint = torch.load(m_path)
#resnet18.load_state_dict(checkpoint['model_state_dict'])

resnet18.to(device)
resnet18.eval()